Level 3: Manufacturing Operations Management Tables

Work Orders

In [8]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Error: Equipment data file {equipment_file} not found.")
        print("Please run the equipment data generation script first.")
        return None

def load_products_data(products_file="data/products.csv"):
    """
    Load the previously generated products data if available
    
    Parameters:
    - products_file: CSV file containing products data
    
    Returns:
    - DataFrame containing the products data or None if not available
    """
    try:
        return pd.read_csv(products_file)
    except FileNotFoundError:
        print(f"Note: Products data file {products_file} not found.")
        print("Work orders will be generated with synthetic product IDs.")
        return None

def load_facilities_data(facilities_file="data/facilities.csv"):
    """
    Load the previously generated facilities data if available
    
    Parameters:
    - facilities_file: CSV file containing facilities data
    
    Returns:
    - DataFrame containing the facilities data or None if not available
    """
    try:
        return pd.read_csv(facilities_file)
    except FileNotFoundError:
        print(f"Note: Facilities data file {facilities_file} not found.")
        print("Work orders will not have facility assignments.")
        return None

def load_customer_orders_data(customer_orders_file="data/customer_orders.csv"):
    """
    Load the previously generated customer orders data if available
    
    Parameters:
    - customer_orders_file: CSV file containing customer orders data
    
    Returns:
    - DataFrame containing the customer orders data or None if not available
    """
    try:
        return pd.read_csv(customer_orders_file)
    except FileNotFoundError:
        print(f"Note: Customer orders data file {customer_orders_file} not found.")
        print("Work orders will be generated with synthetic customer order IDs.")
        return None

def load_production_schedules_data(production_schedules_file="data/production_schedules.csv"):
    """
    Load the previously generated production schedules data if available
    
    Parameters:
    - production_schedules_file: CSV file containing production schedules data
    
    Returns:
    - DataFrame containing the production schedules data or None if not available
    """
    try:
        return pd.read_csv(production_schedules_file)
    except FileNotFoundError:
        print(f"Note: Production schedules data file {production_schedules_file} not found.")
        print("Work orders will be generated with synthetic production schedule IDs.")
        return None

def generate_work_orders(equipment_df, products_df=None, facilities_df=None, 
                        customer_orders_df=None, production_schedules_df=None, 
                        num_work_orders=200, start_time=None, end_time=None,
                        output_file="data/work_orders.csv"):
    """
    Generate synthetic data for the WorkOrders table from ISA-95 Level 3.
    
    Parameters:
    - equipment_df: DataFrame containing equipment data
    - products_df: DataFrame containing products data (optional)
    - facilities_df: DataFrame containing facilities data (optional)
    - customer_orders_df: DataFrame containing customer orders data (optional)
    - production_schedules_df: DataFrame containing production schedules data (optional)
    - num_work_orders: Number of work order records to generate
    - start_time: Start time for work order date range (defaults to 90 days ago)
    - end_time: End time for work order date range (defaults to 30 days in the future)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated work orders data
    """
    if equipment_df is None or len(equipment_df) == 0:
        print("Error: No equipment data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=90)
    if end_time is None:
        end_time = datetime.now() + timedelta(days=30)
    
    # Generate product IDs if not provided
    if products_df is None or len(products_df) == 0:
        print("Generating synthetic product IDs...")
        product_ids = [f"PROD-{uuid.uuid4().hex[:8].upper()}" for _ in range(20)]
    else:
        product_ids = products_df['product_id'].unique().tolist()
    
    # Generate facility IDs if not provided
    if facilities_df is None or len(facilities_df) == 0:
        print("Generating synthetic facility IDs...")
        facility_ids = [f"FAC-{uuid.uuid4().hex[:8].upper()}" for _ in range(5)]
    else:
        facility_ids = facilities_df['facility_id'].unique().tolist()
    
    # Generate customer order IDs if not provided
    if customer_orders_df is None or len(customer_orders_df) == 0:
        print("Generating synthetic customer order IDs...")
        customer_order_ids = [f"CO-{uuid.uuid4().hex[:8].upper()}" for _ in range(50)]
    else:
        customer_order_ids = customer_orders_df['order_id'].unique().tolist()
    
    # Generate production schedule IDs if not provided
    if production_schedules_df is None or len(production_schedules_df) == 0:
        print("Generating synthetic production schedule IDs...")
        production_schedule_ids = [f"PS-{uuid.uuid4().hex[:8].upper()}" for _ in range(10)]
    else:
        production_schedule_ids = production_schedules_df['schedule_id'].unique().tolist()
    
    # Define work order types and their probabilities
    work_order_types = {
        "Production": 0.6,        # Most common
        "Maintenance": 0.2,
        "Quality Check": 0.05,
        "Rework": 0.05,
        "Engineering": 0.05,
        "Cleaning": 0.03,
        "Calibration": 0.02
    }
    
    # Define work order statuses and their transition probabilities
    # Format: {status: {next_status: probability}}
    status_transitions = {
        "Planned": {"Planned": 0.2, "Released": 0.7, "Cancelled": 0.1},
        "Released": {"Released": 0.2, "In Progress": 0.75, "Cancelled": 0.05},
        "In Progress": {"In Progress": 0.3, "Completed": 0.6, "On Hold": 0.1},
        "On Hold": {"On Hold": 0.3, "In Progress": 0.6, "Cancelled": 0.1},
        "Completed": {"Completed": 0.95, "Rework": 0.05},
        "Cancelled": {"Cancelled": 1.0},  # Terminal state
        "Rework": {"Rework": 0.3, "In Progress": 0.7}
    }
    
    # Define priority levels
    priority_levels = [1, 2, 3, 4, 5]  # 1 = highest, 5 = lowest
    priority_weights = [0.1, 0.2, 0.4, 0.2, 0.1]  # Most orders are medium priority
    
    # Define possible units of measurement
    quantity_units = ["kg", "L", "units", "pallets", "boxes", "tons", "m", "m²", "m³", "batches"]
    
    # Generate work order data
    data = {
        "work_order_id": [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_work_orders)],
        "work_order_type": [],
        "product_id": [],
        "planned_quantity": [],
        "actual_quantity": [],
        "quantity_unit": [],
        "planned_start_date": [],
        "actual_start_date": [],
        "planned_end_date": [],
        "actual_end_date": [],
        "status": [],
        "priority": [],
        "customer_order_id": [],
        "production_schedule_id": [],
        "facility_id": []
    }
    
    # Generate a time distribution for work orders (weighted toward recent and near future)
    time_points = []
    time_range_days = (end_time - start_time).days
    
    for _ in range(num_work_orders):
        # Use a beta distribution to weight toward recent and near future
        # Beta(2, 1) will weight toward the future, Beta(1, 2) toward the past
        if random.random() < 0.6:  # 60% of orders are more recent/future
            beta = random.betavariate(2, 1)
        else:
            beta = random.betavariate(1, 2)
        
        days_offset = int(beta * time_range_days)
        time_point = start_time + timedelta(days=days_offset)
        time_points.append(time_point)
    
    # Sort time points to establish a chronological sequence
    time_points.sort()
    
    # Generate data for each work order
    for i in range(num_work_orders):
        # Select work order type (weighted random)
        work_order_type = random.choices(
            list(work_order_types.keys()), 
            weights=list(work_order_types.values())
        )[0]
        data["work_order_type"].append(work_order_type)
        
        # Assign product ID (if applicable)
        if work_order_type in ["Production", "Rework", "Quality Check"]:
            data["product_id"].append(random.choice(product_ids))
        else:
            data["product_id"].append("")  # No product for maintenance, cleaning, etc.
        
        # Generate quantity (if applicable)
        if work_order_type in ["Production", "Rework"]:
            # Production quantities vary by product type, but we'll use a general range
            planned_quantity = random.choice([10, 50, 100, 500, 1000, 5000]) * random.uniform(0.8, 1.2)
            planned_quantity = round(planned_quantity, 1)
            data["planned_quantity"].append(planned_quantity)
            
            # Set unit appropriate for the quantity
            quantity_unit = random.choice(quantity_units)
            data["quantity_unit"].append(quantity_unit)
            
            # Generate actual quantity based on status (to be filled in later)
            data["actual_quantity"].append(0)  # Placeholder
        else:
            data["planned_quantity"].append(0)
            data["quantity_unit"].append("")
            data["actual_quantity"].append(0)
        
        # Set base planned start date from the chronological sequence
        base_date = time_points[i]
        
        # For work orders in the past, determine status through a Markov chain simulation
        current_status = "Planned"  # All work orders start as planned
        current_date = base_date
        
        # Simulate status transitions based on time progression
        while current_date < datetime.now():
            # Get possible next statuses and their probabilities
            next_status_probs = status_transitions[current_status]
            next_statuses = list(next_status_probs.keys())
            next_probs = list(next_status_probs.values())
            
            # Select next status
            next_status = random.choices(next_statuses, weights=next_probs)[0]
            
            # If status changed, advance the date
            if next_status != current_status:
                # Time in a status depends on the status itself
                if current_status == "Planned":
                    days_in_status = random.randint(1, 5)  # 1-5 days in planning
                elif current_status == "Released":
                    days_in_status = random.randint(1, 3)  # 1-3 days released before starting
                elif current_status == "In Progress":
                    days_in_status = random.randint(3, 15)  # 3-15 days in production
                elif current_status == "On Hold":
                    days_in_status = random.randint(2, 10)  # 2-10 days on hold
                else:
                    days_in_status = random.randint(1, 5)  # Default
                
                current_date += timedelta(days=days_in_status)
            else:
                # If status didn't change, just advance a small random amount
                current_date += timedelta(days=random.randint(1, 3))
            
            current_status = next_status
            
            # Stop if we've reached a terminal status
            if current_status in ["Completed", "Cancelled"]:
                break
        
        # Set the final status
        data["status"].append(current_status)
        
        # Set duration based on work order type
        if work_order_type == "Production":
            # Production orders typically take longer
            duration_days = random.randint(5, 20)
        elif work_order_type == "Maintenance":
            duration_days = random.randint(1, 5)
        elif work_order_type == "Cleaning":
            duration_days = random.randint(1, 2)
        elif work_order_type == "Calibration":
            duration_days = random.randint(1, 3)
        else:
            duration_days = random.randint(2, 10)
        
        # Set planned dates
        planned_start_date = base_date
        planned_end_date = planned_start_date + timedelta(days=duration_days)
        
        data["planned_start_date"].append(planned_start_date.strftime("%Y-%m-%d"))
        data["planned_end_date"].append(planned_end_date.strftime("%Y-%m-%d"))
        
        # Set actual dates based on status
        if current_status in ["In Progress", "On Hold", "Completed", "Rework"]:
            # Started but may not be finished
            # Add some variation to actual start date
            start_variation = random.randint(-2, 2)  # +/- 2 days from planned
            actual_start_date = planned_start_date + timedelta(days=start_variation)
            data["actual_start_date"].append(actual_start_date.strftime("%Y-%m-%d"))
            
            if current_status == "Completed":
                # Finished - may be early, on time, or late
                completion_variation = random.choices(
                    [-3, -2, -1, 0, 1, 2, 3, 5, 10],  # Days early(-) or late(+)
                    weights=[0.05, 0.1, 0.15, 0.3, 0.15, 0.1, 0.05, 0.05, 0.05]
                )[0]
                actual_end_date = planned_end_date + timedelta(days=completion_variation)
                data["actual_end_date"].append(actual_end_date.strftime("%Y-%m-%d"))
                
                # Set actual quantity for completed orders
                if data["planned_quantity"][i] > 0:
                    # Actual quantity is usually close to planned, but may vary
                    quantity_variation = random.uniform(0.9, 1.05)  # -10% to +5%
                    actual_quantity = data["planned_quantity"][i] * quantity_variation
                    data["actual_quantity"][i] = round(actual_quantity, 1)
            else:
                # Not finished yet
                data["actual_end_date"].append("")
        else:
            # Not started yet
            data["actual_start_date"].append("")
            data["actual_end_date"].append("")
        
        # Set priority (weighted random)
        data["priority"].append(random.choices(priority_levels, weights=priority_weights)[0])
        
        # Connect to customer order (production orders are more likely to be connected)
        if work_order_type == "Production" and random.random() < 0.8:
            data["customer_order_id"].append(random.choice(customer_order_ids))
        else:
            data["customer_order_id"].append("")
        
        # Connect to production schedule (production orders are more likely to be scheduled)
        if work_order_type == "Production" and random.random() < 0.9:
            data["production_schedule_id"].append(random.choice(production_schedule_ids))
        else:
            data["production_schedule_id"].append("")
        
        # Assign to facility
        data["facility_id"].append(random.choice(facility_ids))
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} work order records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(work_orders_df):
    """
    Display basic statistics about the generated work orders data
    
    Parameters:
    - work_orders_df: DataFrame containing work orders data
    """
    if work_orders_df is None or len(work_orders_df) == 0:
        print("No work orders data to analyze.")
        return
    
    print("\nWork Orders Statistics:")
    print(f"Total work orders: {len(work_orders_df)}")
    
    print("\nWork Order Type Distribution:")
    type_counts = work_orders_df['work_order_type'].value_counts()
    for wo_type, count in type_counts.items():
        print(f"  {wo_type}: {count} ({count/len(work_orders_df)*100:.1f}%)")
    
    print("\nStatus Distribution:")
    status_counts = work_orders_df['status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count} ({count/len(work_orders_df)*100:.1f}%)")
    
    print("\nPriority Distribution:")
    priority_counts = work_orders_df['priority'].value_counts().sort_index()
    for priority, count in priority_counts.items():
        print(f"  Priority {priority}: {count} ({count/len(work_orders_df)*100:.1f}%)")
    
    # Customer order connections
    co_count = work_orders_df['customer_order_id'].apply(lambda x: x != "").sum()
    print(f"\nWork orders linked to customer orders: {co_count} ({co_count/len(work_orders_df)*100:.1f}%)")
    
    # Production schedule connections
    ps_count = work_orders_df['production_schedule_id'].apply(lambda x: x != "").sum()
    print(f"Work orders linked to production schedules: {ps_count} ({ps_count/len(work_orders_df)*100:.1f}%)")
    
    # Production work orders completion rate
    prod_orders = work_orders_df[work_orders_df['work_order_type'] == 'Production']
    if len(prod_orders) > 0:
        completed = prod_orders[prod_orders['status'] == 'Completed']
        print(f"\nProduction work orders completion rate: {len(completed)/len(prod_orders)*100:.1f}%")
        
        # Quantity statistics for production orders
        if len(completed) > 0:
            # Convert to numeric to ensure proper calculations
            completed['planned_quantity'] = pd.to_numeric(completed['planned_quantity'], errors='coerce')
            completed['actual_quantity'] = pd.to_numeric(completed['actual_quantity'], errors='coerce')
            
            # Filter out any NaN values
            valid_quantities = completed.dropna(subset=['planned_quantity', 'actual_quantity'])
            
            if len(valid_quantities) > 0:
                # Calculate yield rate (actual vs. planned)
                valid_quantities['yield_rate'] = valid_quantities['actual_quantity'] / valid_quantities['planned_quantity']
                avg_yield = valid_quantities['yield_rate'].mean() * 100
                print(f"Average yield rate: {avg_yield:.1f}%")
    
    # Date statistics
    print("\nDate Statistics:")
    
    # Convert date strings to datetime objects for comparison
    work_orders_df['planned_start_date'] = pd.to_datetime(work_orders_df['planned_start_date'])
    work_orders_df['planned_end_date'] = pd.to_datetime(work_orders_df['planned_end_date'])
    
    # Filter out empty strings before conversion
    work_orders_df['actual_start_date'] = pd.to_datetime(
        work_orders_df['actual_start_date'].replace('', pd.NaT), errors='coerce'
    )
    work_orders_df['actual_end_date'] = pd.to_datetime(
        work_orders_df['actual_end_date'].replace('', pd.NaT), errors='coerce'
    )
    
    # Calculate planned duration
    work_orders_df['planned_duration'] = (work_orders_df['planned_end_date'] - work_orders_df['planned_start_date']).dt.days
    
    print(f"  Average planned duration: {work_orders_df['planned_duration'].mean():.1f} days")
    
    # Calculate actual duration for completed work orders
    completed = work_orders_df[work_orders_df['status'] == 'Completed'].dropna(subset=['actual_start_date', 'actual_end_date'])
    if len(completed) > 0:
        completed['actual_duration'] = (completed['actual_end_date'] - completed['actual_start_date']).dt.days
        print(f"  Average actual duration: {completed['actual_duration'].mean():.1f} days")
        
        # Calculate on-time delivery
        completed['on_time'] = completed['actual_end_date'] <= completed['planned_end_date']
        on_time_pct = completed['on_time'].mean() * 100
        print(f"  On-time completion rate: {on_time_pct:.1f}%")
    
    # Distribution of work orders over time
    current_month = pd.Timestamp.now().replace(day=1)
    past_months = [current_month - pd.DateOffset(months=i) for i in range(3)]
    future_months = [current_month + pd.DateOffset(months=i) for i in range(1, 3)]
    
    print("\nTime Distribution of Work Orders:")
    for month in past_months + [current_month] + future_months:
        month_start = month
        month_end = month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
        month_orders = work_orders_df[(work_orders_df['planned_start_date'] >= month_start) & 
                                     (work_orders_df['planned_start_date'] <= month_end)]
        month_name = month.strftime("%B %Y")
        print(f"  {month_name}: {len(month_orders)} work orders")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    equipment_df = load_equipment_data()
    products_df = load_products_data()
    facilities_df = load_facilities_data()
    customer_orders_df = load_customer_orders_data()
    production_schedules_df = load_production_schedules_data()
    
    if equipment_df is not None:
        # Generate work orders data
        work_orders_df = generate_work_orders(
            equipment_df,
            products_df,
            facilities_df,
            customer_orders_df,
            production_schedules_df,
            num_work_orders=200,  # Generate 200 work order records
            output_file="data/work_orders.csv"
        )
        
        # Display statistics
        if work_orders_df is not None:
            display_statistics(work_orders_df)
            
            # Display sample data
            print("\nSample work orders data (first 5 records):")
            print(work_orders_df.head(5))

Note: Products data file data/products.csv not found.
Work orders will be generated with synthetic product IDs.
Note: Customer orders data file data/customer_orders.csv not found.
Work orders will be generated with synthetic customer order IDs.
Note: Production schedules data file data/production_schedules.csv not found.
Work orders will be generated with synthetic production schedule IDs.
Generating synthetic product IDs...
Generating synthetic customer order IDs...
Generating synthetic production schedule IDs...
Successfully generated 200 work order records.
Data saved to data/work_orders.csv

Work Orders Statistics:
Total work orders: 200

Work Order Type Distribution:
  Production: 116 (58.0%)
  Maintenance: 43 (21.5%)
  Engineering: 12 (6.0%)
  Rework: 10 (5.0%)
  Quality Check: 10 (5.0%)
  Calibration: 5 (2.5%)
  Cleaning: 4 (2.0%)

Status Distribution:
  Completed: 101 (50.5%)
  Planned: 59 (29.5%)
  Cancelled: 22 (11.0%)
  In Progress: 9 (4.5%)
  Released: 7 (3.5%)
  On Hold: 2

/tmp/ipykernel_114075/2864994044.py:427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed['planned_quantity'] = pd.to_numeric(completed['planned_quantity'], errors='coerce')
/tmp/ipykernel_114075/2864994044.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed['actual_quantity'] = pd.to_numeric(completed['actual_quantity'], errors='coerce')


Material Lots Material Transactions

In [9]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_materials_data(materials_file="data/materials.csv"):
    """
    Load the previously generated materials data if available
    
    Parameters:
    - materials_file: CSV file containing materials data
    
    Returns:
    - DataFrame containing the materials data or None if not available
    """
    try:
        return pd.read_csv(materials_file)
    except FileNotFoundError:
        print(f"Note: Materials data file {materials_file} not found.")
        print("Material lots will be generated with synthetic material IDs.")
        return None

def load_suppliers_data(suppliers_file="data/suppliers.csv"):
    """
    Load the previously generated suppliers data if available
    
    Parameters:
    - suppliers_file: CSV file containing suppliers data
    
    Returns:
    - DataFrame containing the suppliers data or None if not available
    """
    try:
        return pd.read_csv(suppliers_file)
    except FileNotFoundError:
        print(f"Note: Suppliers data file {suppliers_file} not found.")
        print("Material lots will be generated with synthetic supplier IDs.")
        return None

def load_storage_locations_data(storage_locations_file="data/storage_locations.csv"):
    """
    Load the previously generated storage locations data if available
    
    Parameters:
    - storage_locations_file: CSV file containing storage locations data
    
    Returns:
    - DataFrame containing the storage locations data or None if not available
    """
    try:
        return pd.read_csv(storage_locations_file)
    except FileNotFoundError:
        print(f"Note: Storage locations data file {storage_locations_file} not found.")
        print("Material lots will be generated with synthetic storage location IDs.")
        return None

def generate_material_lots(materials_df=None, suppliers_df=None, storage_locations_df=None, 
                          num_lots=300, start_time=None, end_time=None,
                          output_file="data/material_lots.csv"):
    """
    Generate synthetic data for the MaterialLots table from ISA-95 Level 3.
    
    Parameters:
    - materials_df: DataFrame containing materials data (optional)
    - suppliers_df: DataFrame containing suppliers data (optional)
    - storage_locations_df: DataFrame containing storage locations data (optional)
    - num_lots: Number of material lot records to generate
    - start_time: Start time for receipt dates (defaults to 180 days ago)
    - end_time: End time for receipt dates (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated material lots data
    """
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=180)
    if end_time is None:
        end_time = datetime.now()
    
    # Generate material IDs if not provided
    if materials_df is None or len(materials_df) == 0:
        print("Generating synthetic material IDs...")
        material_ids = [f"MAT-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
        
        # Create synthetic material types
        material_types = ["Raw Material", "Packaging", "Intermediate", "Bulk", "Active Ingredient", 
                         "Excipient", "Finished Good", "Component", "Additive", "Catalyst"]
        
        # Assign material types to material IDs
        material_type_map = {}
        for mat_id in material_ids:
            material_type_map[mat_id] = random.choice(material_types)
    else:
        material_ids = materials_df['material_id'].unique().tolist()
        
        # Create material type map from materials_df
        material_type_map = {}
        for _, material in materials_df.iterrows():
            material_type_map[material['material_id']] = material['material_type'] if 'material_type' in material.columns else "Unknown"
    
    # Generate supplier IDs if not provided
    if suppliers_df is None or len(suppliers_df) == 0:
        print("Generating synthetic supplier IDs...")
        supplier_ids = [f"SUP-{uuid.uuid4().hex[:8].upper()}" for _ in range(15)]
    else:
        supplier_ids = suppliers_df['supplier_id'].unique().tolist()
    
    # Generate storage location IDs if not provided
    if storage_locations_df is None or len(storage_locations_df) == 0:
        print("Generating synthetic storage location IDs...")
        storage_location_ids = [f"LOC-{uuid.uuid4().hex[:8].upper()}" for _ in range(20)]
    else:
        storage_location_ids = storage_locations_df['location_id'].unique().tolist()
    
    # Define quality status options and their probabilities
    quality_statuses = {
        "Released": 0.7,          # Most lots are released
        "Under Quarantine": 0.1,
        "Rejected": 0.05,
        "Pending Test": 0.1,
        "Hold": 0.03,
        "Expired": 0.02
    }
    
    # Define possible units of measurement
    quantity_units = {
        "Raw Material": ["kg", "tons", "L", "m³", "drums"],
        "Packaging": ["units", "rolls", "boxes", "pallets", "sheets"],
        "Intermediate": ["kg", "L", "batches", "drums", "totes"],
        "Bulk": ["kg", "L", "m³", "tons", "batches"],
        "Active Ingredient": ["kg", "g", "mg", "L", "batches"],
        "Excipient": ["kg", "g", "L", "drums", "bags"],
        "Finished Good": ["units", "boxes", "pallets", "cases", "bottles"],
        "Component": ["units", "pieces", "sets", "packages", "boxes"],
        "Additive": ["kg", "g", "L", "drums", "bags"],
        "Catalyst": ["kg", "g", "L", "containers", "vials"]
    }
    
    # Generate data structure
    data = {
        "lot_id": [f"LOT-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_lots)],
        "material_id": [],
        "lot_quantity": [],
        "quantity_unit": [],
        "status": [],
        "creation_date": [],
        "expiration_date": [],
        "supplier_id": [],
        "supplier_lot_id": [],
        "receipt_date": [],
        "storage_location_id": [],
        "quality_status": [],
        "cost_per_unit": [],
        "parent_lot_id": []
    }
    
    # Track lots for potential parent-child relationships
    all_lots = data["lot_id"].copy()
    potential_parents = random.sample(all_lots, int(len(all_lots) * 0.2))  # 20% can be parents
    
    # Generate receipt dates distributed over the time range
    receipt_dates = []
    time_range_days = (end_time - start_time).days
    
    for _ in range(num_lots):
        # Use a beta distribution to weight toward more recent receipts
        if random.random() < 0.7:  # 70% of lots are more recent
            beta = random.betavariate(2, 1)
        else:
            beta = random.betavariate(1, 2)
        
        days_offset = int(beta * time_range_days)
        receipt_date = start_time + timedelta(days=days_offset)
        receipt_dates.append(receipt_date)
    
    # Sort receipt dates (older to newer)
    receipt_dates.sort()
    
    # Generate data for each material lot
    for i in range(num_lots):
        # Select material ID
        material_id = random.choice(material_ids)
        data["material_id"].append(material_id)
        
        # Get material type (for appropriate unit selection)
        material_type = material_type_map.get(material_id, "Raw Material")
        
        # Select quantity unit based on material type
        if material_type in quantity_units:
            unit = random.choice(quantity_units[material_type])
        else:
            unit = random.choice(["kg", "units", "L", "pallets", "pieces"])
        
        data["quantity_unit"].append(unit)
        
        # Generate lot quantity (based on unit)
        if unit in ["kg", "L"]:
            # Typically ordered in hundreds or thousands
            quantity = random.choice([100, 200, 500, 1000, 2000, 5000]) * random.uniform(0.8, 1.2)
        elif unit in ["g", "mg", "ml"]:
            # Small quantities for fine materials
            quantity = random.choice([100, 500, 1000, 5000, 10000]) * random.uniform(0.8, 1.2)
        elif unit in ["tons", "m³"]:
            # Bulk materials in smaller quantities
            quantity = random.choice([1, 2, 5, 10, 20, 50]) * random.uniform(0.8, 1.2)
        elif unit in ["units", "pieces", "bottles"]:
            # Discrete items often in multiples of packaging sizes
            quantity = random.choice([100, 500, 1000, 5000, 10000, 25000])
        elif unit in ["pallets", "cases", "boxes"]:
            # Packaged goods in smaller counts
            quantity = random.choice([5, 10, 20, 50, 100]) * random.uniform(0.8, 1.2)
        else:
            # Default quantity
            quantity = random.choice([10, 50, 100, 500, 1000]) * random.uniform(0.8, 1.2)
        
        data["lot_quantity"].append(round(quantity, 2))
        
        # Set receipt date from the generated distribution
        receipt_date = receipt_dates[i]
        data["receipt_date"].append(receipt_date.strftime("%Y-%m-%d"))
        
        # Creation date is typically shortly before receipt (manufacturing date at supplier)
        manufacturing_lead_time = random.randint(1, 30)  # 1-30 days lead time
        creation_date = receipt_date - timedelta(days=manufacturing_lead_time)
        data["creation_date"].append(creation_date.strftime("%Y-%m-%d"))
        
        # Set expiration date based on material type
        if material_type == "Raw Material":
            shelf_life_days = random.randint(365, 1825)  # 1-5 years
        elif material_type == "Active Ingredient":
            shelf_life_days = random.randint(180, 1095)  # 6 months to 3 years
        elif material_type in ["Intermediate", "Bulk"]:
            shelf_life_days = random.randint(90, 365)  # 3 months to 1 year
        elif material_type == "Finished Good":
            shelf_life_days = random.randint(180, 730)  # 6 months to 2 years
        elif material_type == "Packaging":
            shelf_life_days = random.randint(730, 3650)  # 2-10 years
        else:
            shelf_life_days = random.randint(365, 1095)  # 1-3 years
        
        expiration_date = creation_date + timedelta(days=shelf_life_days)
        data["expiration_date"].append(expiration_date.strftime("%Y-%m-%d"))
        
        # Determine status (based on quantity remaining)
        if random.random() < 0.7:  # 70% are active inventory
            data["status"].append("Active")
        elif random.random() < 0.5:  # Half of the remainder are consumed
            data["status"].append("Consumed")
        else:  # The rest are reserved or in process
            data["status"].append(random.choice(["Reserved", "In Process"]))
        
        # Assign supplier (raw materials and packaging always have suppliers)
        if material_type in ["Raw Material", "Packaging", "Active Ingredient", "Excipient", "Component"]:
            data["supplier_id"].append(random.choice(supplier_ids))
            # Generate supplier's lot ID
            data["supplier_lot_id"].append(f"{random.choice(['L', 'B', 'S'])}{random.randint(10000, 99999)}")
        else:
            # Internal materials may not have external suppliers
            if random.random() < 0.3:  # 30% chance of having supplier even for internal materials
                data["supplier_id"].append(random.choice(supplier_ids))
                data["supplier_lot_id"].append(f"{random.choice(['L', 'B', 'S'])}{random.randint(10000, 99999)}")
            else:
                data["supplier_id"].append("")
                data["supplier_lot_id"].append("")
        
        # Assign storage location
        if data["status"][i] in ["Active", "Reserved"]:
            data["storage_location_id"].append(random.choice(storage_location_ids))
        else:
            # Consumed or in-process materials may not have a storage location
            data["storage_location_id"].append("")
        
        # Set quality status (weighted random)
        data["quality_status"].append(
            random.choices(list(quality_statuses.keys()), weights=list(quality_statuses.values()))[0]
        )
        
        # Generate cost per unit (based on material type)
        if material_type == "Active Ingredient":
            # Expensive materials
            cost = random.uniform(100, 5000)
        elif material_type in ["Raw Material", "Excipient", "Catalyst"]:
            # Moderate cost materials
            cost = random.uniform(5, 100)
        elif material_type in ["Packaging", "Component"]:
            # Lower cost materials
            cost = random.uniform(0.5, 10)
        elif material_type == "Finished Good":
            # Higher value products
            cost = random.uniform(20, 500)
        else:
            # Default cost range
            cost = random.uniform(1, 50)
        
        data["cost_per_unit"].append(round(cost, 2))
        
        # Determine parent lot (if any)
        # Intermediate, Bulk, and Finished Good materials are more likely to have parent lots
        if (material_type in ["Intermediate", "Bulk", "Finished Good"] and 
            data["lot_id"][i] not in potential_parents and 
            random.random() < 0.4):  # 40% chance for applicable materials
            
            # Find suitable parents (created before this lot)
            earlier_lots = [all_lots[j] for j in range(i) if receipt_dates[j] < receipt_date]
            if earlier_lots:
                parent_id = random.choice(earlier_lots)
                data["parent_lot_id"].append(parent_id)
            else:
                data["parent_lot_id"].append("")
        else:
            data["parent_lot_id"].append("")
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} material lot records.")
    print(f"Data saved to {output_file}")
    
    return df

def generate_material_transactions(material_lots_df, work_orders_df=None, num_transactions=500,
                                  start_time=None, end_time=None, output_file="data/material_transactions.csv"):
    """
    Generate synthetic data for the MaterialTransactions table.
    
    Parameters:
    - material_lots_df: DataFrame containing material lots data
    - work_orders_df: DataFrame containing work orders data (optional)
    - num_transactions: Number of transaction records to generate
    - start_time: Start time for transaction dates (defaults to 180 days ago)
    - end_time: End time for transaction dates (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated material transactions data
    """
    if material_lots_df is None or len(material_lots_df) == 0:
        print("Error: No material lots data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=180)
    if end_time is None:
        end_time = datetime.now()
    
    # Get storage location IDs from material lots
    storage_locations = material_lots_df['storage_location_id'].unique()
    storage_locations = [loc for loc in storage_locations if pd.notna(loc) and loc != ""]
    
    if not storage_locations:
        print("Warning: No storage locations found in material lots data.")
        storage_locations = [f"LOC-{uuid.uuid4().hex[:8].upper()}" for _ in range(5)]
    
    # Get work order IDs if available
    if work_orders_df is not None and len(work_orders_df) > 0:
        work_order_ids = work_orders_df['work_order_id'].unique().tolist()
    else:
        work_order_ids = []
    
    # Define transaction types and their probabilities
    transaction_types = {
        "Receipt": 0.25,          # Initial receipt from supplier
        "Issue": 0.3,             # Issue to production
        "Return": 0.05,           # Return from production
        "Transfer": 0.2,          # Move between locations
        "Adjustment": 0.1,        # Inventory adjustment
        "Consumption": 0.05,      # Material consumed
        "Scrapping": 0.05         # Disposal of material
    }
    
    # Define possible reasons for each transaction type
    transaction_reasons = {
        "Receipt": ["Initial Receipt", "Purchase Order", "Vendor Delivery", "Stock Replenishment", "Contract Manufacturing"],
        "Issue": ["Production Order", "Work Order", "Batch Production", "Line Replenishment", "Process Requirement"],
        "Return": ["Excess Material", "Quality Issue", "Process Change", "Order Cancellation", "Wrong Material"],
        "Transfer": ["Storage Optimization", "Staging for Production", "Quarantine", "Relocation", "Consolidation"],
        "Adjustment": ["Cycle Count", "Inventory Audit", "Quantity Correction", "System Reconciliation", "Physical Count"],
        "Consumption": ["Material Used", "Process Consumption", "Batch Completion", "Manufacturing Process", "Test Samples"],
        "Scrapping": ["Quality Rejection", "Expired Material", "Damaged Goods", "Contamination", "Obsolete Material"]
    }
    
    # Generate transaction data
    data = {
        "transaction_id": [f"TRAN-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_transactions)],
        "transaction_type": [],
        "lot_id": [],
        "timestamp": [],
        "quantity": [],
        "from_location_id": [],
        "to_location_id": [],
        "work_order_id": [],
        "batch_id": [],
        "operator_id": [],
        "transaction_reason": [],
        "reference_document": []
    }
    
    # Generate operator IDs
    operator_ids = [f"OP-{uuid.uuid4().hex[:6].upper()}" for _ in range(10)]
    
    # Generate batch IDs (synthetic)
    batch_ids = [f"BATCH-{uuid.uuid4().hex[:8].upper()}" for _ in range(20)]
    
    # Generate document reference patterns
    po_pattern = "PO-{}"
    wo_pattern = "WO-{}"
    gr_pattern = "GR-{}"
    adj_pattern = "ADJ-{}"
    
    # Generate transactions
    active_lots = material_lots_df[material_lots_df['status'].isin(['Active', 'Reserved', 'In Process'])]['lot_id'].tolist()
    consumed_lots = material_lots_df[material_lots_df['status'] == 'Consumed']['lot_id'].tolist()
    all_lots = material_lots_df['lot_id'].tolist()
    
    # Generate timestamps distributed over the time range
    time_range_minutes = int((end_time - start_time).total_seconds() / 60)
    timestamps = []
    
    for _ in range(num_transactions):
        random_minutes = random.randint(0, time_range_minutes)
        timestamp = start_time + timedelta(minutes=random_minutes)
        timestamps.append(timestamp)
    
    # Sort timestamps (older to newer)
    timestamps.sort()
    
    # Now generate transaction data
    for i in range(num_transactions):
        # Determine transaction type (weighted random)
        transaction_type = random.choices(list(transaction_types.keys()), 
                                        weights=list(transaction_types.values()))[0]
        data["transaction_type"].append(transaction_type)
        
        # Set timestamp
        data["timestamp"].append(timestamps[i].strftime("%Y-%m-%d %H:%M:%S"))
        
        # Select lot based on transaction type
        if transaction_type == "Receipt":
            # Use any lot (as if it's being received)
            lot_id = random.choice(all_lots)
        elif transaction_type in ["Issue", "Transfer", "Return", "Adjustment"]:
            # Use active lots
            if active_lots:
                lot_id = random.choice(active_lots)
            else:
                lot_id = random.choice(all_lots)
        elif transaction_type in ["Consumption", "Scrapping"]:
            # Prefer consumed lots for consistency, but can use any
            if consumed_lots and random.random() < 0.7:
                lot_id = random.choice(consumed_lots)
            elif active_lots:
                lot_id = random.choice(active_lots)
            else:
                lot_id = random.choice(all_lots)
        else:
            lot_id = random.choice(all_lots)
        
        data["lot_id"].append(lot_id)
        
        # Get lot information
        lot_info = material_lots_df[material_lots_df['lot_id'] == lot_id].iloc[0]
        total_quantity = lot_info['lot_quantity']
        
        # Determine transaction quantity
        if transaction_type == "Receipt":
            # Receipt is typically the full quantity
            quantity = total_quantity
        elif transaction_type == "Issue":
            # Issue is typically a portion or all of the quantity
            quantity = total_quantity * random.uniform(0.1, 1.0)
        elif transaction_type == "Return":
            # Return is typically a smaller portion
            quantity = total_quantity * random.uniform(0.05, 0.3)
        elif transaction_type == "Transfer":
            # Transfer is typically the full quantity
            quantity = total_quantity
        elif transaction_type == "Adjustment":
            # Adjustment can be positive or negative
            if random.random() < 0.5:
                # Positive adjustment
                quantity = total_quantity * random.uniform(0.01, 0.1)
            else:
                # Negative adjustment
                quantity = -total_quantity * random.uniform(0.01, 0.1)
        elif transaction_type == "Consumption":
            # Consumption is typically a large portion or all
            quantity = total_quantity * random.uniform(0.5, 1.0)
        elif transaction_type == "Scrapping":
            # Scrapping can be a portion or all
            quantity = total_quantity * random.uniform(0.1, 1.0)
        
        data["quantity"].append(round(quantity, 2))
        
        # Set location information based on transaction type
        if transaction_type == "Receipt":
            # From supplier (blank) to storage
            data["from_location_id"].append("")
            data["to_location_id"].append(random.choice(storage_locations))
        elif transaction_type == "Issue":
            # From storage to production (can be blank)
            data["from_location_id"].append(lot_info['storage_location_id'] if pd.notna(lot_info['storage_location_id']) else random.choice(storage_locations))
            data["to_location_id"].append("")  # Issued to production, not a storage location
        elif transaction_type == "Return":
            # From production (blank) to storage
            data["from_location_id"].append("")
            data["to_location_id"].append(lot_info['storage_location_id'] if pd.notna(lot_info['storage_location_id']) else random.choice(storage_locations))
        elif transaction_type == "Transfer":
            # From one storage location to another
            from_loc = lot_info['storage_location_id'] if pd.notna(lot_info['storage_location_id']) else random.choice(storage_locations)
            # Ensure to_location is different from from_location
            available_to_locs = [loc for loc in storage_locations if loc != from_loc]
            to_loc = random.choice(available_to_locs) if available_to_locs else random.choice(storage_locations)
            data["from_location_id"].append(from_loc)
            data["to_location_id"].append(to_loc)
        elif transaction_type == "Adjustment":
            # Adjustment happens in the current location
            data["from_location_id"].append(lot_info['storage_location_id'] if pd.notna(lot_info['storage_location_id']) else random.choice(storage_locations))
            data["to_location_id"].append("")
        elif transaction_type in ["Consumption", "Scrapping"]:
            # From storage to nowhere (consumed/scrapped)
            data["from_location_id"].append(lot_info['storage_location_id'] if pd.notna(lot_info['storage_location_id']) else random.choice(storage_locations))
            data["to_location_id"].append("")
        
        # Associate with work order if applicable
        if transaction_type in ["Issue", "Consumption"] and work_order_ids and random.random() < 0.8:
            # 80% chance of having a work order for production-related transactions
            data["work_order_id"].append(random.choice(work_order_ids))
        elif transaction_type == "Return" and work_order_ids and random.random() < 0.6:
            # 60% chance of having a work order for returns
            data["work_order_id"].append(random.choice(work_order_ids))
        else:
            data["work_order_id"].append("")
        
        # Associate with batch if applicable
        if transaction_type in ["Issue", "Consumption", "Return"] and random.random() < 0.7:
            # 70% chance of having a batch for production-related transactions
            data["batch_id"].append(random.choice(batch_ids))
        else:
            data["batch_id"].append("")
        
        # Set operator
        data["operator_id"].append(random.choice(operator_ids))
        
        # Set transaction reason
        if transaction_type in transaction_reasons:
            reason = random.choice(transaction_reasons[transaction_type])
        else:
            reason = "Standard Transaction"
        
        data["transaction_reason"].append(reason)
        
        # Generate reference document
        if transaction_type == "Receipt":
            ref_doc = po_pattern.format(random.randint(10000, 99999))
        elif transaction_type in ["Issue", "Consumption"]:
            if data["work_order_id"][i]:
                ref_doc = wo_pattern.format(data["work_order_id"][i].split('-')[-1])
            else:
                ref_doc = wo_pattern.format(random.randint(10000, 99999))
        elif transaction_type == "Adjustment":
            ref_doc = adj_pattern.format(random.randint(10000, 99999))
        elif transaction_type == "Transfer":
            ref_doc = gr_pattern.format(random.randint(10000, 99999))
        else:
            ref_doc = f"DOC-{random.randint(10000, 99999)}"
        
        data["reference_document"].append(ref_doc)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} material transaction records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(material_lots_df, material_transactions_df=None):
    """
    Display basic statistics about the generated material lots and transactions data
    
    Parameters:
    - material_lots_df: DataFrame containing material lots data
    - material_transactions_df: DataFrame containing material transactions data (optional)
    """
    if material_lots_df is None or len(material_lots_df) == 0:
        print("No material lots data to analyze.")
        return
    
    print("\nMaterial Lots Statistics:")
    print(f"Total material lots: {len(material_lots_df)}")
    
    print("\nStatus Distribution:")
    status_counts = material_lots_df['status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count} ({count/len(material_lots_df)*100:.1f}%)")
    
    print("\nQuality Status Distribution:")
    quality_counts = material_lots_df['quality_status'].value_counts()
    for status, count in quality_counts.items():
        print(f"  {status}: {count} ({count/len(material_lots_df)*100:.1f}%)")
    
    print("\nQuantity Units Distribution:")
    unit_counts = material_lots_df['quantity_unit'].value_counts().head(10)
    for unit, count in unit_counts.items():
        print(f"  {unit}: {count} ({count/len(material_lots_df)*100:.1f}%)")
    
    # Supplier connections
    supplier_count = material_lots_df['supplier_id'].apply(lambda x: x != "").sum()
    print(f"\nLots with supplier association: {supplier_count} ({supplier_count/len(material_lots_df)*100:.1f}%)")
    
    # Storage location connections
    location_count = material_lots_df['storage_location_id'].apply(lambda x: x != "").sum()
    print(f"Lots with storage location: {location_count} ({location_count/len(material_lots_df)*100:.1f}%)")
    
    # Parent lot relationships
    parent_count = material_lots_df['parent_lot_id'].apply(lambda x: x != "").sum()
    print(f"Lots with parent lot: {parent_count} ({parent_count/len(material_lots_df)*100:.1f}%)")
    
    # Quantity and cost statistics
    print("\nQuantity Statistics:")
    print(f"  Total quantity: {material_lots_df['lot_quantity'].sum():.1f} (in various units)")
    print(f"  Average lot size: {material_lots_df['lot_quantity'].mean():.1f}")
    print(f"  Min lot size: {material_lots_df['lot_quantity'].min():.1f}")
    print(f"  Max lot size: {material_lots_df['lot_quantity'].max():.1f}")
    
    print("\nCost Statistics:")
    print(f"  Average cost per unit: ${material_lots_df['cost_per_unit'].mean():.2f}")
    print(f"  Min cost per unit: ${material_lots_df['cost_per_unit'].min():.2f}")
    print(f"  Max cost per unit: ${material_lots_df['cost_per_unit'].max():.2f}")
    
    # Age analysis
    material_lots_df['creation_date'] = pd.to_datetime(material_lots_df['creation_date'])
    material_lots_df['expiration_date'] = pd.to_datetime(material_lots_df['expiration_date'])
    
    material_lots_df['shelf_life_days'] = (material_lots_df['expiration_date'] - material_lots_df['creation_date']).dt.days
    
    print("\nShelf Life Statistics:")
    print(f"  Average shelf life: {material_lots_df['shelf_life_days'].mean():.1f} days")
    print(f"  Min shelf life: {material_lots_df['shelf_life_days'].min()} days")
    print(f"  Max shelf life: {material_lots_df['shelf_life_days'].max()} days")
    
    # Calculate remaining shelf life
    now = pd.Timestamp.now()
    material_lots_df['remaining_days'] = (material_lots_df['expiration_date'] - now).dt.days
    
    # Only consider active lots
    active_lots = material_lots_df[material_lots_df['status'] == 'Active']
    if len(active_lots) > 0:
        print("\nRemaining Shelf Life (Active Lots):")
        print(f"  Average remaining shelf life: {active_lots['remaining_days'].mean():.1f} days")
        print(f"  Min remaining shelf life: {active_lots['remaining_days'].min()} days")
        print(f"  Max remaining shelf life: {active_lots['remaining_days'].max()} days")
        
        # Expired lots
        expired_count = len(active_lots[active_lots['remaining_days'] <= 0])
        print(f"  Expired lots still active: {expired_count} ({expired_count/len(active_lots)*100:.1f}% of active lots)")
        
        # Near expiry (less than 30 days)
        near_expiry = len(active_lots[(active_lots['remaining_days'] > 0) & (active_lots['remaining_days'] <= 30)])
        print(f"  Lots expiring within 30 days: {near_expiry} ({near_expiry/len(active_lots)*100:.1f}% of active lots)")
    
    # Material transactions statistics (if available)
    if material_transactions_df is not None and len(material_transactions_df) > 0:
        print("\nMaterial Transactions Statistics:")
        print(f"Total transactions: {len(material_transactions_df)}")
        
        print("\nTransaction Type Distribution:")
        trans_type_counts = material_transactions_df['transaction_type'].value_counts()
        for trans_type, count in trans_type_counts.items():
            print(f"  {trans_type}: {count} ({count/len(material_transactions_df)*100:.1f}%)")
        
        # Work order connections
        wo_count = material_transactions_df['work_order_id'].apply(lambda x: x != "").sum()
        print(f"\nTransactions with work order: {wo_count} ({wo_count/len(material_transactions_df)*100:.1f}%)")
        
        # Batch connections
        batch_count = material_transactions_df['batch_id'].apply(lambda x: x != "").sum()
        print(f"Transactions with batch: {batch_count} ({batch_count/len(material_transactions_df)*100:.1f}%)")
        
        # Quantity statistics
        print("\nTransaction Quantity Statistics:")
        print(f"  Average transaction quantity: {material_transactions_df['quantity'].mean():.1f}")
        print(f"  Min transaction quantity: {material_transactions_df['quantity'].min():.1f}")
        print(f"  Max transaction quantity: {material_transactions_df['quantity'].max():.1f}")
        
        # Time distribution
        material_transactions_df['timestamp'] = pd.to_datetime(material_transactions_df['timestamp'])
        
        # Group by month
        material_transactions_df['month'] = material_transactions_df['timestamp'].dt.to_period('M')
        monthly_counts = material_transactions_df['month'].value_counts().sort_index()
        
        print("\nMonthly Transaction Distribution:")
        for month, count in monthly_counts.items():
            print(f"  {month}: {count} transactions")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    materials_df = load_materials_data()
    suppliers_df = load_suppliers_data()
    storage_locations_df = load_storage_locations_data()
    
    # Try to load work orders for transactions
    try:
        work_orders_df = pd.read_csv("data/work_orders.csv")
    except FileNotFoundError:
        print("Note: Work orders data file not found.")
        work_orders_df = None
    
    # Generate material lots data
    material_lots_df = generate_material_lots(
        materials_df,
        suppliers_df,
        storage_locations_df,
        num_lots=300,  # Generate 300 material lot records
        output_file="data/material_lots.csv"
    )
    
    # Generate material transactions if lots were created successfully
    if material_lots_df is not None:
        material_transactions_df = generate_material_transactions(
            material_lots_df,
            work_orders_df,
            num_transactions=500,  # Generate 500 transaction records
            output_file="data/material_transactions.csv"
        )
    else:
        material_transactions_df = None
    
    # Display statistics
    if material_lots_df is not None:
        display_statistics(material_lots_df, material_transactions_df)
        
        # Display sample data
        print("\nSample material lots data (first 5 records):")
        print(material_lots_df.head(5))
        
        if material_transactions_df is not None:
            print("\nSample material transactions data (first 5 records):")
            print(material_transactions_df.head(5))

Note: Materials data file data/materials.csv not found.
Material lots will be generated with synthetic material IDs.
Note: Suppliers data file data/suppliers.csv not found.
Material lots will be generated with synthetic supplier IDs.
Note: Storage locations data file data/storage_locations.csv not found.
Material lots will be generated with synthetic storage location IDs.
Generating synthetic material IDs...
Generating synthetic supplier IDs...
Generating synthetic storage location IDs...
Successfully generated 300 material lot records.
Data saved to data/material_lots.csv
Successfully generated 500 material transaction records.
Data saved to data/material_transactions.csv

Material Lots Statistics:
Total material lots: 300

Status Distribution:
  Active: 189 (63.0%)
  Consumed: 63 (21.0%)
  In Process: 28 (9.3%)
  Reserved: 20 (6.7%)

Quality Status Distribution:
  Released: 207 (69.0%)
  Under Quarantine: 34 (11.3%)
  Pending Test: 28 (9.3%)
  Rejected: 15 (5.0%)
  Hold: 9 (3.0%)
  E

Material Consumptions

In [10]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_material_lots_data(material_lots_file="data/material_lots.csv"):
    """
    Load the previously generated material lots data
    
    Parameters:
    - material_lots_file: CSV file containing material lots data
    
    Returns:
    - DataFrame containing the material lots data
    """
    try:
        return pd.read_csv(material_lots_file)
    except FileNotFoundError:
        print(f"Error: Material lots data file {material_lots_file} not found.")
        print("Please run the material lots data generation script first.")
        return None

def load_batches_data(batches_file="data/batches.csv"):
    """
    Load the previously generated batches data if available
    
    Parameters:
    - batches_file: CSV file containing batches data
    
    Returns:
    - DataFrame containing the batches data or None if not available
    """
    try:
        return pd.read_csv(batches_file)
    except FileNotFoundError:
        print(f"Note: Batches data file {batches_file} not found.")
        print("Material consumption will be generated with synthetic batch IDs.")
        return None

def load_work_orders_data(work_orders_file="data/work_orders.csv"):
    """
    Load the previously generated work orders data if available
    
    Parameters:
    - work_orders_file: CSV file containing work orders data
    
    Returns:
    - DataFrame containing the work orders data or None if not available
    """
    try:
        return pd.read_csv(work_orders_file)
    except FileNotFoundError:
        print(f"Note: Work orders data file {work_orders_file} not found.")
        print("Material consumption will be generated with synthetic work order IDs.")
        return None

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data if available
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data or None if not available
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Note: Equipment data file {equipment_file} not found.")
        print("Material consumption will be generated with synthetic equipment IDs.")
        return None

def load_batch_execution_data(batch_execution_file="data/batch_execution.csv"):
    """
    Load the previously generated batch execution data if available
    
    Parameters:
    - batch_execution_file: CSV file containing batch execution data
    
    Returns:
    - DataFrame containing the batch execution data or None if not available
    """
    try:
        return pd.read_csv(batch_execution_file)
    except FileNotFoundError:
        print(f"Note: Batch execution data file {batch_execution_file} not found.")
        print("Material consumption will be generated with synthetic batch step IDs.")
        return None

def load_personnel_data(personnel_file="data/personnel.csv"):
    """
    Load the previously generated personnel data if available
    
    Parameters:
    - personnel_file: CSV file containing personnel data
    
    Returns:
    - DataFrame containing the personnel data or None if not available
    """
    try:
        return pd.read_csv(personnel_file)
    except FileNotFoundError:
        print(f"Note: Personnel data file {personnel_file} not found.")
        print("Material consumption will be generated with synthetic operator IDs.")
        return None

def generate_material_consumption(material_lots_df, batches_df=None, work_orders_df=None, 
                                equipment_df=None, batch_execution_df=None, personnel_df=None,
                                num_consumptions=400, start_time=None, end_time=None,
                                output_file="data/material_consumption.csv"):
    """
    Generate synthetic data for the MaterialConsumption table from ISA-95 Level 3.
    
    Parameters:
    - material_lots_df: DataFrame containing material lots data
    - batches_df: DataFrame containing batches data (optional)
    - work_orders_df: DataFrame containing work orders data (optional)
    - equipment_df: DataFrame containing equipment data (optional)
    - batch_execution_df: DataFrame containing batch execution data (optional)
    - personnel_df: DataFrame containing personnel data (optional)
    - num_consumptions: Number of material consumption records to generate
    - start_time: Start time for consumption dates (defaults to 180 days ago)
    - end_time: End time for consumption dates (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated material consumption data
    """
    if material_lots_df is None or len(material_lots_df) == 0:
        print("Error: No material lots data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=180)
    if end_time is None:
        end_time = datetime.now()
    
    # Filter material lots that can be consumed (active or in process)
    consumable_lots = material_lots_df[
        (material_lots_df['status'].isin(['Active', 'In Process', 'Consumed']))
    ]
    
    if len(consumable_lots) == 0:
        print("Warning: No consumable material lots found. Using all available lots.")
        consumable_lots = material_lots_df
    
    # Get batch IDs if available, otherwise generate synthetic ones
    if batches_df is not None and len(batches_df) > 0:
        batch_ids = batches_df['batch_id'].unique().tolist()
    else:
        print("Generating synthetic batch IDs...")
        batch_ids = [f"BATCH-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
    
    # Get work order IDs if available, otherwise generate synthetic ones
    if work_orders_df is not None and len(work_orders_df) > 0:
        work_order_ids = work_orders_df['work_order_id'].unique().tolist()
    else:
        print("Generating synthetic work order IDs...")
        work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
    
    # Get equipment IDs if available, otherwise generate synthetic ones
    if equipment_df is not None and len(equipment_df) > 0:
        equipment_ids = equipment_df['equipment_id'].unique().tolist()
    else:
        print("Generating synthetic equipment IDs...")
        equipment_ids = [f"EQ-{uuid.uuid4().hex[:8].upper()}" for _ in range(20)]
    
    # Get batch step IDs if available, otherwise generate synthetic ones
    if batch_execution_df is not None and len(batch_execution_df) > 0:
        batch_step_ids = batch_execution_df['step_id'].unique().tolist()
    else:
        print("Generating synthetic batch step IDs...")
        batch_step_ids = [f"STEP-{uuid.uuid4().hex[:8].upper()}" for _ in range(50)]
    
    # Get operator IDs if available, otherwise generate synthetic ones
    if personnel_df is not None and len(personnel_df) > 0:
        operator_ids = personnel_df['personnel_id'].unique().tolist()
    else:
        print("Generating synthetic operator IDs...")
        operator_ids = [f"OP-{uuid.uuid4().hex[:6].upper()}" for _ in range(15)]
    
    # Generate consumption data
    data = {
        "consumption_id": [f"CONS-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_consumptions)],
        "lot_id": [],
        "batch_id": [],
        "work_order_id": [],
        "timestamp": [],
        "quantity": [],
        "unit": [],
        "equipment_id": [],
        "step_id": [],
        "operator_id": [],
        "planned_consumption": [],
        "consumption_variance": []
    }
    
    # Generate timestamps distributed over the time range
    time_range_minutes = int((end_time - start_time).total_seconds() / 60)
    timestamps = []
    
    for _ in range(num_consumptions):
        random_minutes = random.randint(0, time_range_minutes)
        timestamp = start_time + timedelta(minutes=random_minutes)
        timestamps.append(timestamp)
    
    # Sort timestamps (older to newer)
    timestamps.sort()
    
    # Generate data for each consumption record
    for i in range(num_consumptions):
        # Select a material lot to consume
        if len(consumable_lots) > 0:
            lot = consumable_lots.sample(1).iloc[0]
            data["lot_id"].append(lot['lot_id'])
            
            # Use the lot's unit
            unit = lot['quantity_unit']
            data["unit"].append(unit)
            
            # Maximum consumption is the lot quantity
            max_consumption = float(lot['lot_quantity'])
            
            # Typical consumption is a portion of the lot
            typical_consumption = max_consumption * random.uniform(0.05, 0.9)
        else:
            # Fallback if no lots are available
            data["lot_id"].append(f"LOT-{uuid.uuid4().hex[:8].upper()}")
            unit = random.choice(["kg", "L", "units", "g", "ml", "pieces"])
            data["unit"].append(unit)
            max_consumption = random.uniform(100, 5000)
            typical_consumption = max_consumption * random.uniform(0.05, 0.9)
        
        # Set timestamp
        data["timestamp"].append(timestamps[i].strftime("%Y-%m-%d %H:%M:%S"))
        
        # Assign to batch and work order
        # Consumption records typically have both, but we'll allow some variation
        if random.random() < 0.9:  # 90% have batch
            data["batch_id"].append(random.choice(batch_ids))
        else:
            data["batch_id"].append("")
            
        if random.random() < 0.8:  # 80% have work order
            data["work_order_id"].append(random.choice(work_order_ids))
        else:
            data["work_order_id"].append("")
        
        # Assign equipment
        data["equipment_id"].append(random.choice(equipment_ids))
        
        # Assign batch step
        if random.random() < 0.7:  # 70% have specific step
            data["step_id"].append(random.choice(batch_step_ids))
        else:
            data["step_id"].append("")
        
        # Assign operator
        data["operator_id"].append(random.choice(operator_ids))
        
        # Generate consumption quantity
        # Actual consumption has some variance from planned
        planned_consumption = round(typical_consumption, 2)
        data["planned_consumption"].append(planned_consumption)
        
        # Actual consumption varies from planned
        variation_pct = random.normalvariate(0, 0.05)  # Normal distribution around 0 with 5% std dev
        actual_consumption = planned_consumption * (1 + variation_pct)
        actual_consumption = round(min(max_consumption, max(0, actual_consumption)), 2)
        data["quantity"].append(actual_consumption)
        
        # Calculate variance
        variance = actual_consumption - planned_consumption
        data["consumption_variance"].append(round(variance, 2))
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} material consumption records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(material_consumption_df):
    """
    Display basic statistics about the generated material consumption data
    
    Parameters:
    - material_consumption_df: DataFrame containing material consumption data
    """
    if material_consumption_df is None or len(material_consumption_df) == 0:
        print("No material consumption data to analyze.")
        return
    
    print("\nMaterial Consumption Statistics:")
    print(f"Total consumption records: {len(material_consumption_df)}")
    
    # Batch connections
    batch_count = material_consumption_df['batch_id'].apply(lambda x: x != "").sum()
    print(f"\nConsumption records with batch: {batch_count} ({batch_count/len(material_consumption_df)*100:.1f}%)")
    
    # Work order connections
    wo_count = material_consumption_df['work_order_id'].apply(lambda x: x != "").sum()
    print(f"Consumption records with work order: {wo_count} ({wo_count/len(material_consumption_df)*100:.1f}%)")
    
    # Step connections
    step_count = material_consumption_df['step_id'].apply(lambda x: x != "").sum()
    print(f"Consumption records with batch step: {step_count} ({step_count/len(material_consumption_df)*100:.1f}%)")
    
    # Quantity statistics
    print("\nConsumption Quantity Statistics:")
    print(f"  Total quantity consumed: {material_consumption_df['quantity'].sum():.1f} (in various units)")
    print(f"  Average consumption quantity: {material_consumption_df['quantity'].mean():.1f}")
    print(f"  Min consumption quantity: {material_consumption_df['quantity'].min():.1f}")
    print(f"  Max consumption quantity: {material_consumption_df['quantity'].max():.1f}")
    
    # Planned vs Actual statistics
    print("\nPlanned vs Actual Consumption:")
    print(f"  Total planned consumption: {material_consumption_df['planned_consumption'].sum():.1f}")
    print(f"  Total actual consumption: {material_consumption_df['quantity'].sum():.1f}")
    print(f"  Total variance: {material_consumption_df['consumption_variance'].sum():.1f}")
    
    # Calculate average absolute variance percentage
    material_consumption_df['variance_pct'] = abs(material_consumption_df['consumption_variance'] / material_consumption_df['planned_consumption'] * 100)
    avg_variance_pct = material_consumption_df['variance_pct'].mean()
    print(f"  Average absolute variance: {avg_variance_pct:.1f}%")
    
    # Distribution of variances
    over_consumption = material_consumption_df[material_consumption_df['consumption_variance'] > 0]
    under_consumption = material_consumption_df[material_consumption_df['consumption_variance'] < 0]
    on_target = material_consumption_df[material_consumption_df['consumption_variance'] == 0]
    
    print("\nConsumption Variance Distribution:")
    print(f"  Over-consumption: {len(over_consumption)} records ({len(over_consumption)/len(material_consumption_df)*100:.1f}%)")
    print(f"  Under-consumption: {len(under_consumption)} records ({len(under_consumption)/len(material_consumption_df)*100:.1f}%)")
    print(f"  On-target: {len(on_target)} records ({len(on_target)/len(material_consumption_df)*100:.1f}%)")
    
    # Time distribution
    material_consumption_df['timestamp'] = pd.to_datetime(material_consumption_df['timestamp'])
    
    # Group by month
    material_consumption_df['month'] = material_consumption_df['timestamp'].dt.to_period('M')
    monthly_counts = material_consumption_df['month'].value_counts().sort_index()
    
    print("\nMonthly Consumption Distribution:")
    for month, count in monthly_counts.items():
        print(f"  {month}: {count} consumption records")
    
    # Unit distribution
    unit_counts = material_consumption_df['unit'].value_counts()
    print("\nUnit of Measurement Distribution:")
    for unit, count in unit_counts.items():
        print(f"  {unit}: {count} records ({count/len(material_consumption_df)*100:.1f}%)")
    
    # Material lot frequency analysis
    lot_frequency = material_consumption_df['lot_id'].value_counts()
    print("\nMaterial Lot Usage Frequency:")
    print(f"  Average usage per lot: {lot_frequency.mean():.1f} times")
    print(f"  Max usage per lot: {lot_frequency.max()} times")
    print(f"  Single-use lots: {(lot_frequency == 1).sum()} lots")
    print(f"  Multi-use lots: {(lot_frequency > 1).sum()} lots")
    
    # Equipment distribution
    equipment_counts = material_consumption_df['equipment_id'].value_counts().head(10)
    print("\nTop Equipment Usage:")
    for equipment_id, count in equipment_counts.items():
        print(f"  {equipment_id}: {count} consumption records")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    material_lots_df = load_material_lots_data()
    batches_df = load_batches_data()
    work_orders_df = load_work_orders_data()
    equipment_df = load_equipment_data()
    batch_execution_df = load_batch_execution_data()
    personnel_df = load_personnel_data()
    
    if material_lots_df is not None:
        # Generate material consumption data
        material_consumption_df = generate_material_consumption(
            material_lots_df,
            batches_df,
            work_orders_df,
            equipment_df,
            batch_execution_df,
            personnel_df,
            num_consumptions=400,  # Generate 400 material consumption records
            output_file="data/material_consumption.csv"
        )
        
        # Display statistics
        if material_consumption_df is not None:
            display_statistics(material_consumption_df)
            
            # Display sample data
            print("\nSample material consumption data (first 5 records):")
            print(material_consumption_df.head(5))

Note: Personnel data file data/personnel.csv not found.
Material consumption will be generated with synthetic operator IDs.
Generating synthetic operator IDs...
Successfully generated 400 material consumption records.
Data saved to data/material_consumption.csv

Material Consumption Statistics:
Total consumption records: 400

Consumption records with batch: 360 (90.0%)
Consumption records with work order: 310 (77.5%)
Consumption records with batch step: 270 (67.5%)

Consumption Quantity Statistics:
  Total quantity consumed: 282133.2 (in various units)
  Average consumption quantity: 705.3
  Min consumption quantity: 0.2
  Max consumption quantity: 19105.0

Planned vs Actual Consumption:
  Total planned consumption: 285244.0
  Total actual consumption: 282133.2
  Total variance: -3110.8
  Average absolute variance: 4.3%

Consumption Variance Distribution:
  Over-consumption: 201 records (50.2%)
  Under-consumption: 193 records (48.2%)
  On-target: 6 records (1.5%)

Monthly Consumption 

Quality Test Table

In [11]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_material_lots_data(material_lots_file="data/material_lots.csv"):
    """
    Load the previously generated material lots data
    
    Parameters:
    - material_lots_file: CSV file containing material lots data
    
    Returns:
    - DataFrame containing the material lots data
    """
    try:
        return pd.read_csv(material_lots_file)
    except FileNotFoundError:
        print(f"Error: Material lots data file {material_lots_file} not found.")
        print("Please run the material lots data generation script first.")
        return None

def load_products_data(products_file="data/products.csv"):
    """
    Load the previously generated products data if available
    
    Parameters:
    - products_file: CSV file containing products data
    
    Returns:
    - DataFrame containing the products data or None if not available
    """
    try:
        return pd.read_csv(products_file)
    except FileNotFoundError:
        print(f"Note: Products data file {products_file} not found.")
        print("Quality tests will be generated with synthetic product IDs.")
        return None

def load_batches_data(batches_file="data/batches.csv"):
    """
    Load the previously generated batches data if available
    
    Parameters:
    - batches_file: CSV file containing batches data
    
    Returns:
    - DataFrame containing the batches data or None if not available
    """
    try:
        return pd.read_csv(batches_file)
    except FileNotFoundError:
        print(f"Note: Batches data file {batches_file} not found.")
        print("Quality tests will be generated with synthetic batch IDs.")
        return None

def load_work_orders_data(work_orders_file="data/work_orders.csv"):
    """
    Load the previously generated work orders data if available
    
    Parameters:
    - work_orders_file: CSV file containing work orders data
    
    Returns:
    - DataFrame containing the work orders data or None if not available
    """
    try:
        return pd.read_csv(work_orders_file)
    except FileNotFoundError:
        print(f"Note: Work orders data file {work_orders_file} not found.")
        print("Quality tests will be generated with synthetic work order IDs.")
        return None

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data if available
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data or None if not available
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Note: Equipment data file {equipment_file} not found.")
        print("Quality tests will be generated with synthetic equipment IDs.")
        return None

def load_personnel_data(personnel_file="data/personnel.csv"):
    """
    Load the previously generated personnel data if available
    
    Parameters:
    - personnel_file: CSV file containing personnel data
    
    Returns:
    - DataFrame containing the personnel data or None if not available
    """
    try:
        return pd.read_csv(personnel_file)
    except FileNotFoundError:
        print(f"Note: Personnel data file {personnel_file} not found.")
        print("Quality tests will be generated with synthetic inspector IDs.")
        return None

def generate_quality_tests(material_lots_df, products_df=None, batches_df=None, 
                          work_orders_df=None, equipment_df=None, personnel_df=None,
                          num_tests=500, start_time=None, end_time=None,
                          output_file="data/quality_tests.csv"):
    """
    Generate synthetic data for the QualityTests table from ISA-95 Level 3.
    
    Parameters:
    - material_lots_df: DataFrame containing material lots data
    - products_df: DataFrame containing products data (optional)
    - batches_df: DataFrame containing batches data (optional)
    - work_orders_df: DataFrame containing work orders data (optional)
    - equipment_df: DataFrame containing equipment data (optional)
    - personnel_df: DataFrame containing personnel data (optional)
    - num_tests: Number of quality test records to generate
    - start_time: Start time for test dates (defaults to 180 days ago)
    - end_time: End time for test dates (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated quality tests data
    """
    if material_lots_df is None or len(material_lots_df) == 0:
        print("Error: No material lots data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=180)
    if end_time is None:
        end_time = datetime.now()
    
    # Get product IDs if available, otherwise generate synthetic ones
    if products_df is not None and len(products_df) > 0:
        product_ids = products_df['product_id'].unique().tolist()
    else:
        print("Generating synthetic product IDs...")
        product_ids = [f"PROD-{uuid.uuid4().hex[:8].upper()}" for _ in range(20)]
    
    # Get batch IDs if available, otherwise generate synthetic ones
    if batches_df is not None and len(batches_df) > 0:
        batch_ids = batches_df['batch_id'].unique().tolist()
    else:
        print("Generating synthetic batch IDs...")
        batch_ids = [f"BATCH-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
    
    # Get work order IDs if available, otherwise generate synthetic ones
    if work_orders_df is not None and len(work_orders_df) > 0:
        work_order_ids = work_orders_df['work_order_id'].unique().tolist()
    else:
        print("Generating synthetic work order IDs...")
        work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
    
    # Get equipment IDs (specifically for test equipment) if available, otherwise generate synthetic ones
    if equipment_df is not None and len(equipment_df) > 0:
        test_equipment_ids = equipment_df.sample(min(10, len(equipment_df)))['equipment_id'].tolist()
    else:
        print("Generating synthetic test equipment IDs...")
        test_equipment_ids = [f"EQ-{uuid.uuid4().hex[:8].upper()}" for _ in range(10)]
    
    # Get inspector IDs if available, otherwise generate synthetic ones
    if personnel_df is not None and len(personnel_df) > 0:
        inspector_ids = personnel_df.sample(min(20, len(personnel_df)))['personnel_id'].tolist()
    else:
        print("Generating synthetic inspector IDs...")
        inspector_ids = [f"PERS-{uuid.uuid4().hex[:8].upper()}" for _ in range(15)]
    
    # Define test types and their probabilities
    test_types = {
        "Chemical Analysis": 0.2,
        "Physical Test": 0.15,
        "Visual Inspection": 0.15,
        "Microbiological": 0.1,
        "Stability Test": 0.05,
        "Dimensional Check": 0.1,
        "Functional Test": 0.1,
        "Identification Test": 0.05,
        "Impurity Test": 0.05,
        "Release Testing": 0.05
    }
    
    # Define test methods for each test type
    test_methods = {
        "Chemical Analysis": ["HPLC", "GC", "MS", "IR Spectroscopy", "UV-Vis Spectroscopy", "Titration", "pH Measurement", "Conductivity"],
        "Physical Test": ["Viscosity", "Density", "Particle Size", "Hardness", "Tensile Strength", "Melting Point", "Dissolution", "Friability"],
        "Visual Inspection": ["Appearance", "Color", "Clarity", "Foreign Particles", "Visible Defects", "Packaging Integrity"],
        "Microbiological": ["Total Plate Count", "Microbial Enumeration", "Sterility", "Endotoxin", "Bioburden", "Antimicrobial Effectiveness"],
        "Stability Test": ["Accelerated Stability", "Long-term Stability", "Photostability", "Temperature Cycling", "Stress Testing"],
        "Dimensional Check": ["Height", "Width", "Diameter", "Thickness", "Weight", "Volume", "Surface Area"],
        "Functional Test": ["Performance Test", "Operational Check", "Power Consumption", "Response Time", "Load Test", "Durability"],
        "Identification Test": ["IR Identity", "Chemical Identity", "Chromatographic Identity", "Spectral Comparison"],
        "Impurity Test": ["Related Substances", "Residual Solvents", "Heavy Metals", "Organic Impurities", "Inorganic Impurities"],
        "Release Testing": ["Final Product Test", "Certificate of Analysis", "Conformance Test", "Quality Control Release"]
    }
    
    # Define test parameters and their specifications for each test type
    test_parameters = {
        "Chemical Analysis": {
            "Assay Content": {"unit": "%", "target": 100.0, "range": 5.0},
            "pH": {"unit": "pH units", "target": 7.0, "range": 1.0},
            "Residual Solvent": {"unit": "ppm", "target": 0.0, "range": 1000.0, "upper_only": True},
            "Active Ingredient": {"unit": "mg/mL", "target": 10.0, "range": 1.0},
            "Conductivity": {"unit": "µS/cm", "target": 100.0, "range": 50.0}
        },
        "Physical Test": {
            "Viscosity": {"unit": "cP", "target": 1000.0, "range": 200.0},
            "Density": {"unit": "g/cm³", "target": 1.05, "range": 0.1},
            "Particle Size": {"unit": "µm", "target": 50.0, "range": 10.0},
            "Hardness": {"unit": "kP", "target": 12.0, "range": 3.0},
            "Dissolution": {"unit": "%", "target": 85.0, "range": 15.0, "lower_only": True}
        },
        "Visual Inspection": {
            "Appearance": {"unit": "score", "target": 5.0, "range": 1.0, "lower_only": True},
            "Color Conformity": {"unit": "score", "target": 5.0, "range": 1.0, "lower_only": True},
            "Visible Defects": {"unit": "count", "target": 0.0, "range": 3.0, "upper_only": True},
            "Label Quality": {"unit": "score", "target": 5.0, "range": 1.0, "lower_only": True}
        },
        "Microbiological": {
            "Total Aerobic Count": {"unit": "CFU/g", "target": 0.0, "range": 1000.0, "upper_only": True},
            "E. coli": {"unit": "CFU/g", "target": 0.0, "range": 0.0, "upper_only": True}, # Zero tolerance
            "Yeast & Mold": {"unit": "CFU/g", "target": 0.0, "range": 100.0, "upper_only": True},
            "Salmonella": {"unit": "presence", "target": 0.0, "range": 0.0, "upper_only": True} # Pass/fail
        },
        "Stability Test": {
            "Potency": {"unit": "%", "target": 100.0, "range": 10.0},
            "Degradation Products": {"unit": "%", "target": 0.0, "range": 2.0, "upper_only": True},
            "pH Change": {"unit": "pH units", "target": 0.0, "range": 1.0, "absolute": True}
        },
        "Dimensional Check": {
            "Length": {"unit": "mm", "target": 100.0, "range": 1.0},
            "Width": {"unit": "mm", "target": 50.0, "range": 0.5},
            "Height": {"unit": "mm", "target": 25.0, "range": 0.5},
            "Weight": {"unit": "g", "target": 500.0, "range": 15.0}
        },
        "Functional Test": {
            "Operation Time": {"unit": "seconds", "target": 60.0, "range": 10.0},
            "Power Output": {"unit": "W", "target": 1000.0, "range": 100.0},
            "Efficiency": {"unit": "%", "target": 95.0, "range": 5.0},
            "Response Time": {"unit": "ms", "target": 100.0, "range": 20.0}
        },
        "Identification Test": {
            "Identity": {"unit": "match", "target": 1.0, "range": 0.0, "lower_only": True}, # Pass/fail
            "Purity": {"unit": "%", "target": 99.0, "range": 1.0, "lower_only": True}
        },
        "Impurity Test": {
            "Individual Impurity": {"unit": "%", "target": 0.0, "range": 0.5, "upper_only": True},
            "Total Impurities": {"unit": "%", "target": 0.0, "range": 2.0, "upper_only": True},
            "Heavy Metals": {"unit": "ppm", "target": 0.0, "range": 10.0, "upper_only": True}
        },
        "Release Testing": {
            "Final Potency": {"unit": "%", "target": 100.0, "range": 5.0},
            "Final Impurities": {"unit": "%", "target": 0.0, "range": 2.0, "upper_only": True},
            "Uniformity": {"unit": "%RSD", "target": 0.0, "range": 5.0, "upper_only": True}
        }
    }
    
    # Generate data structure
    data = {
        "test_id": [f"TEST-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_tests)],
        "test_type": [],
        "test_method": [],
        "sample_id": [],
        "product_id": [],
        "lot_id": [],
        "batch_id": [],
        "work_order_id": [],
        "timestamp": [],
        "parameter_name": [],
        "specification_target": [],
        "specification_lower_limit": [],
        "specification_upper_limit": [],
        "actual_value": [],
        "unit": [],
        "test_result": [],
        "test_equipment_id": [],
        "analyst_id": [],
        "retest_flag": [],
        "notes": []
    }
    
    # Generate timestamps distributed over the time range
    time_range_minutes = int((end_time - start_time).total_seconds() / 60)
    timestamps = []
    
    for _ in range(num_tests):
        random_minutes = random.randint(0, time_range_minutes)
        timestamp = start_time + timedelta(minutes=random_minutes)
        timestamps.append(timestamp)
    
    # Sort timestamps (older to newer)
    timestamps.sort()
    
    # Generate data for each test record
    for i in range(num_tests):
        # Select test type (weighted random)
        test_type = random.choices(list(test_types.keys()), weights=list(test_types.values()))[0]
        data["test_type"].append(test_type)
        
        # Select test method for this type
        test_method = random.choice(test_methods[test_type])
        data["test_method"].append(test_method)
        
        # Generate sample ID
        data["sample_id"].append(f"S{random.randint(100000, 999999)}")
        
        # Decide what's being tested: material lot, product, or both
        test_target = random.choice(["lot", "product", "both"])
        
        if test_target in ["lot", "both"]:
            # Test is for a material lot
            lot_id = random.choice(material_lots_df['lot_id'].tolist())
            data["lot_id"].append(lot_id)
            
            # Get associated material info if possible
            try:
                lot_info = material_lots_df[material_lots_df['lot_id'] == lot_id].iloc[0]
                material_id = lot_info.get('material_id', "")
            except (IndexError, KeyError):
                material_id = ""
        else:
            data["lot_id"].append("")
            material_id = ""
            
        if test_target in ["product", "both"]:
            # Test is for a product
            data["product_id"].append(random.choice(product_ids))
        else:
            data["product_id"].append("")
        
        # Associate with batch and work order
        if random.random() < 0.7:  # 70% associated with batch
            data["batch_id"].append(random.choice(batch_ids))
        else:
            data["batch_id"].append("")
            
        if random.random() < 0.6:  # 60% associated with work order
            data["work_order_id"].append(random.choice(work_order_ids))
        else:
            data["work_order_id"].append("")
        
        # Set timestamp
        data["timestamp"].append(timestamps[i].strftime("%Y-%m-%d %H:%M:%S"))
        
        # Select test parameter for this type
        parameter_name = random.choice(list(test_parameters[test_type].keys()))
        data["parameter_name"].append(parameter_name)
        
        # Get parameter specs
        param_specs = test_parameters[test_type][parameter_name]
        target_value = param_specs["target"]
        range_value = param_specs["range"]
        unit = param_specs["unit"]
        
        # Set specification limits
        upper_only = param_specs.get("upper_only", False)
        lower_only = param_specs.get("lower_only", False)
        absolute = param_specs.get("absolute", False)
        
        if upper_only:
            # Only upper limit (max allowed)
            lower_limit = ""
            upper_limit = target_value + range_value
        elif lower_only:
            # Only lower limit (min required)
            lower_limit = target_value - range_value
            upper_limit = ""
        else:
            # Both limits
            lower_limit = target_value - range_value
            upper_limit = target_value + range_value
        
        # Special handling for zero or near-zero targets
        if abs(target_value) < 0.001 and not absolute:
            lower_limit = 0.0
        
        data["specification_target"].append(target_value)
        data["specification_lower_limit"].append(lower_limit)
        data["specification_upper_limit"].append(upper_limit)
        data["unit"].append(unit)
        
        # Generate actual test value (normally distributed around target with occasional outliers)
        if random.random() < 0.05:  # 5% chance of outlier
            # Generate outlier value
            if upper_only:
                # For upper-only specs, generate occasional high outliers
                actual_value = target_value + (range_value * random.uniform(1.1, 2.0))
            elif lower_only:
                # For lower-only specs, generate occasional low outliers
                actual_value = target_value - (range_value * random.uniform(1.1, 2.0))
            else:
                # For two-sided specs, generate outliers on either side
                if random.random() < 0.5:
                    actual_value = target_value + (range_value * random.uniform(1.1, 1.5))
                else:
                    actual_value = target_value - (range_value * random.uniform(1.1, 1.5))
        else:
            # Generate normal value (normally distributed around target)
            std_dev = range_value / 3.0  # 3-sigma rule: most values within spec
            actual_value = random.normalvariate(target_value, std_dev)
        
        # Handle special cases
        if unit == "presence" or unit == "match":
            # These are pass/fail tests, actual value should be 0 or 1
            if random.random() < 0.95:  # 95% pass rate
                actual_value = 1 if unit == "match" else 0  # Match=1 is good, Presence=0 is good
            else:
                actual_value = 0 if unit == "match" else 1
        
        # Round actual value based on the unit precision
        if "%" in unit:
            actual_value = round(actual_value, 1)  # One decimal for percentages
        elif unit in ["g/cm³", "pH units"]:
            actual_value = round(actual_value, 2)  # Two decimals for density, pH
        elif unit in ["µm", "mg/mL", "ppm"]:
            actual_value = round(actual_value, 1)  # One decimal for small measurements
        else:
            actual_value = round(actual_value, 2)  # Default precision
        
        data["actual_value"].append(actual_value)
        
        # Determine test result
        if upper_only and upper_limit != "":
            test_result = "Pass" if actual_value <= upper_limit else "Fail"
        elif lower_only and lower_limit != "":
            test_result = "Pass" if actual_value >= lower_limit else "Fail"
        elif upper_limit != "" and lower_limit != "":
            test_result = "Pass" if lower_limit <= actual_value <= upper_limit else "Fail"
        else:
            # Default for unusual cases
            test_result = "Pass" if random.random() < 0.95 else "Fail"
        
        data["test_result"].append(test_result)
        
        # Assign test equipment
        data["test_equipment_id"].append(random.choice(test_equipment_ids))
        
        # Assign analyst/inspector
        data["analyst_id"].append(random.choice(inspector_ids))
        
        # Set retest flag (more likely for failed tests)
        if test_result == "Fail":
            retest_flag = random.random() < 0.7  # 70% of failures get retested
        else:
            retest_flag = random.random() < 0.05  # 5% of passes get retested
        
        data["retest_flag"].append(retest_flag)
        
        # Generate notes (more detailed for failures)
        if test_result == "Fail":
            notes_options = [
                f"Out of specification. Retest authorized.",
                f"Value exceeds {parameter_name} limit. Investigation required.",
                f"Failed {test_method} test. Checking calibration.",
                f"OOS result confirmed on duplicate test.",
                f"Deviation reported, sample under investigation."
            ]
        else:
            notes_options = [
                f"Result within specification.",
                f"Test completed successfully.",
                f"Verified against standard.",
                f"",  # Empty note for many passing tests
                f""
            ]
        
        data["notes"].append(random.choice(notes_options))
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} quality test records.")
    print(f"Data saved to {output_file}")
    
    return df

def generate_quality_events(quality_tests_df, material_lots_df=None, batches_df=None, 
                          equipment_df=None, process_areas_df=None, personnel_df=None,
                          num_events=100, output_file="data/quality_events.csv"):
    """
    Generate synthetic data for the QualityEvents table based on quality tests.
    
    Parameters:
    - quality_tests_df: DataFrame containing quality tests data
    - material_lots_df: DataFrame containing material lots data (optional)
    - batches_df: DataFrame containing batches data (optional)
    - equipment_df: DataFrame containing equipment data (optional)
    - process_areas_df: DataFrame containing process areas data (optional)
    - personnel_df: DataFrame containing personnel data (optional)
    - num_events: Number of quality event records to generate
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated quality events data
    """
    if quality_tests_df is None or len(quality_tests_df) == 0:
        print("Error: No quality tests data available.")
        return None
    
    # Get failed tests as potential sources for quality events
    failed_tests = quality_tests_df[quality_tests_df['test_result'] == "Fail"]
    
    if len(failed_tests) == 0:
        print("Warning: No failed quality tests found. Generating generic quality events.")
        test_based_events = False
    else:
        test_based_events = True
    
    # Get material lot IDs
    if material_lots_df is not None and len(material_lots_df) > 0:
        lot_ids = material_lots_df['lot_id'].unique().tolist()
    else:
        lot_ids = quality_tests_df['lot_id'].unique().tolist()
        lot_ids = [lot for lot in lot_ids if lot]  # Remove empty strings
    
    # Get batch IDs
    if batches_df is not None and len(batches_df) > 0:
        batch_ids = batches_df['batch_id'].unique().tolist()
    else:
        batch_ids = quality_tests_df['batch_id'].unique().tolist()
        batch_ids = [batch for batch in batch_ids if batch]  # Remove empty strings
    
    # Get equipment IDs
    if equipment_df is not None and len(equipment_df) > 0:
        equipment_ids = equipment_df['equipment_id'].unique().tolist()
    else:
        equipment_ids = quality_tests_df['test_equipment_id'].unique().tolist()
    
    # Get process area IDs if available, otherwise generate synthetic ones
    if process_areas_df is not None and len(process_areas_df) > 0:
        area_ids = process_areas_df['area_id'].unique().tolist()
    else:
        print("Generating synthetic process area IDs...")
        area_ids = [f"AREA-{uuid.uuid4().hex[:8].upper()}" for _ in range(10)]
    
    # Get personnel IDs
    if personnel_df is not None and len(personnel_df) > 0:
        personnel_ids = personnel_df['personnel_id'].unique().tolist()
    else:
        personnel_ids = quality_tests_df['analyst_id'].unique().tolist()
    
    # Define event types and their probabilities
    event_types = {
        "Deviation": 0.3,
        "Non-conformance": 0.25,
        "Quality Incident": 0.2,
        "OOS Result": 0.15,
        "Complaint": 0.1
    }
    
    # Define severity levels
    severity_levels = [1, 2, 3, 4, 5]  # 1 = minor, 5 = critical
    severity_weights = [0.3, 0.3, 0.2, 0.15, 0.05]  # Most events are lower severity
    
    # Define status options
    event_statuses = ["Open", "Under Investigation", "Corrective Action", "Closed", "Canceled"]
    
    # Generate data structure
    data = {
        "event_id": [f"QE-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_events)],
        "event_type": [],
        "severity": [],
        "description": [],
        "detection_date": [],
        "status": [],
        "product_id": [],
        "lot_id": [],
        "batch_id": [],
        "equipment_id": [],
        "area_id": [],
        "detected_by": [],
        "assignee": [],
        "root_cause": [],
        "corrective_action": [],
        "closure_date": []
    }
    
    # Generate a timeframe for events (past 180 days)
    start_time = datetime.now() - timedelta(days=180)
    end_time = datetime.now()
    
    # Root cause categories
    root_causes = [
        "Process Deviation", "Equipment Malfunction", "Human Error", "Material Quality", 
        "Environmental Factors", "Contamination", "Documentation Error", "Training Issue",
        "Supplier Quality", "Unknown"
    ]
    
    # Corrective action templates
    corrective_actions = [
        "Retrain Personnel", "Update Procedure", "Equipment Maintenance", "Material Replacement",
        "Process Modification", "Enhanced Monitoring", "Supplier Audit", "CAPA Implementation",
        "Preventive Maintenance", "Additional Testing"
    ]
    
    # Generate data for each quality event
    for i in range(num_events):
        # Select event type (weighted random)
        event_type = random.choices(list(event_types.keys()), weights=list(event_types.values()))[0]
        data["event_type"].append(event_type)
        
        # Set severity (weighted random)
        severity = random.choices(severity_levels, weights=severity_weights)[0]
        data["severity"].append(severity)
        
        # Determine if event is based on failed test
        if test_based_events and random.random() < 0.7:  # 70% of events based on failed tests
            # Select a random failed test
            test = failed_tests.sample(1).iloc[0]
            
            # Use test information
            test_id = test['test_id']
            parameter = test['parameter_name']
            test_type = test['test_type']
            actual_value = test['actual_value']
            target_value = test['specification_target']
            unit = test['unit']
            
            # Create description based on test
            description = f"{event_type} for {parameter} in {test_type} test. Actual: {actual_value} {unit}, Target: {target_value} {unit}. Test ID: {test_id}"
            
            # Link to the same entities
            product_id = test['product_id']
            lot_id = test['lot_id']
            batch_id = test['batch_id']
            equipment_id = test['test_equipment_id']
            detected_by = test['analyst_id']
            
        else:
            # Generate generic quality event
            templates = [
                f"{event_type}: {random.choice(['High', 'Low', 'Out of range', 'Unexpected'])} {random.choice(['viscosity', 'content', 'weight', 'appearance', 'dissolution'])} result",
                f"{event_type}: {random.choice(['Foreign material', 'Contamination', 'Incorrect label', 'Missing component', 'Wrong color'])} detected",
                f"{event_type}: {random.choice(['Process parameter', 'Equipment', 'Material', 'Documentation'])} {random.choice(['issue', 'failure', 'deviation', 'error'])}"
            ]
            description = random.choice(templates)
            
            # Random associations
            product_id = random.choice(quality_tests_df['product_id'].unique().tolist()) if random.random() < 0.7 else ""
            lot_id = random.choice(lot_ids) if lot_ids and random.random() < 0.8 else ""
            batch_id = random.choice(batch_ids) if batch_ids and random.random() < 0.7 else ""
            equipment_id = random.choice(equipment_ids) if random.random() < 0.6 else ""
            detected_by = random.choice(personnel_ids)
        
        data["description"].append(description)
        data["product_id"].append(product_id)
        data["lot_id"].append(lot_id)
        data["batch_id"].append(batch_id)
        data["equipment_id"].append(equipment_id)
        data["detected_by"].append(detected_by)
        
        # Generate detection date
        detection_date = start_time + timedelta(days=random.randint(0, 180))
        data["detection_date"].append(detection_date.strftime("%Y-%m-%d"))
        
        # Assign process area
        data["area_id"].append(random.choice(area_ids) if random.random() < 0.8 else "")
        
        # Assign different person as assignee
        available_assignees = [p for p in personnel_ids if p != detected_by]
        assignee = random.choice(available_assignees) if available_assignees else random.choice(personnel_ids)
        data["assignee"].append(assignee)
        
        # Determine status (time-dependent)
        days_since_detection = (datetime.now() - detection_date).days
        
        if days_since_detection < 7:
            # Recent events are typically still open
            status = random.choice(["Open", "Under Investigation"])
        elif days_since_detection < 30:
            # Medium-term events are in progress
            status = random.choice(["Under Investigation", "Corrective Action", "Open"])
        else:
            # Older events are likely closed
            status = random.choice(["Closed", "Closed", "Closed", "Corrective Action", "Canceled"])
        
        data["status"].append(status)
        
        # Set root cause and corrective action (only for investigated/closed events)
        if status in ["Corrective Action", "Closed"]:
            data["root_cause"].append(random.choice(root_causes))
            data["corrective_action"].append(random.choice(corrective_actions))
        else:
            data["root_cause"].append("")
            data["corrective_action"].append("")
        
        # Set closure date (only for closed events)
        if status == "Closed":
            # Closure date is after detection date
            min_closure_delay = 3  # Minimum 3 days to close
            max_closure_delay = min(90, days_since_detection)  # Up to 90 days or available time
            closure_days = random.randint(min_closure_delay, max(min_closure_delay, max_closure_delay))
            closure_date = detection_date + timedelta(days=closure_days)
            data["closure_date"].append(closure_date.strftime("%Y-%m-%d"))
        else:
            data["closure_date"].append("")
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} quality event records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(quality_tests_df, quality_events_df=None):
    """
    Display basic statistics about the generated quality tests and events data
    
    Parameters:
    - quality_tests_df: DataFrame containing quality tests data
    - quality_events_df: DataFrame containing quality events data (optional)
    """
    if quality_tests_df is None or len(quality_tests_df) == 0:
        print("No quality tests data to analyze.")
        return
    
    print("\nQuality Tests Statistics:")
    print(f"Total quality tests: {len(quality_tests_df)}")
    
    # Test type distribution
    print("\nTest Type Distribution:")
    type_counts = quality_tests_df['test_type'].value_counts()
    for test_type, count in type_counts.items():
        print(f"  {test_type}: {count} ({count/len(quality_tests_df)*100:.1f}%)")
    
    # Test result distribution
    print("\nTest Result Distribution:")
    result_counts = quality_tests_df['test_result'].value_counts()
    for result, count in result_counts.items():
        print(f"  {result}: {count} ({count/len(quality_tests_df)*100:.1f}%)")
    
    # Retest flag distribution
    retest_count = quality_tests_df['retest_flag'].sum()
    print(f"\nTests flagged for retest: {retest_count} ({retest_count/len(quality_tests_df)*100:.1f}%)")
    
    # Failed test distribution by type
    failed_tests = quality_tests_df[quality_tests_df['test_result'] == "Fail"]
    if len(failed_tests) > 0:
        print("\nFailed Tests by Type:")
        failed_type_counts = failed_tests['test_type'].value_counts()
        for test_type, count in failed_type_counts.items():
            type_total = type_counts[test_type]
            print(f"  {test_type}: {count}/{type_total} ({count/type_total*100:.1f}% failure rate)")
    
    # Connections to other entities
    lot_count = quality_tests_df['lot_id'].apply(lambda x: x != "").sum()
    product_count = quality_tests_df['product_id'].apply(lambda x: x != "").sum()
    batch_count = quality_tests_df['batch_id'].apply(lambda x: x != "").sum()
    wo_count = quality_tests_df['work_order_id'].apply(lambda x: x != "").sum()
    
    print("\nTest Associations:")
    print(f"  Tests associated with lots: {lot_count} ({lot_count/len(quality_tests_df)*100:.1f}%)")
    print(f"  Tests associated with products: {product_count} ({product_count/len(quality_tests_df)*100:.1f}%)")
    print(f"  Tests associated with batches: {batch_count} ({batch_count/len(quality_tests_df)*100:.1f}%)")
    print(f"  Tests associated with work orders: {wo_count} ({wo_count/len(quality_tests_df)*100:.1f}%)")
    
    # Time distribution
    quality_tests_df['timestamp'] = pd.to_datetime(quality_tests_df['timestamp'])
    
    # Group by month
    quality_tests_df['month'] = quality_tests_df['timestamp'].dt.to_period('M')
    monthly_counts = quality_tests_df['month'].value_counts().sort_index()
    
    print("\nMonthly Test Distribution:")
    for month, count in monthly_counts.items():
        print(f"  {month}: {count} tests")
    
    # Quality events statistics (if available)
    if quality_events_df is not None and len(quality_events_df) > 0:
        print("\nQuality Events Statistics:")
        print(f"Total quality events: {len(quality_events_df)}")
        
        # Event type distribution
        print("\nEvent Type Distribution:")
        event_type_counts = quality_events_df['event_type'].value_counts()
        for event_type, count in event_type_counts.items():
            print(f"  {event_type}: {count} ({count/len(quality_events_df)*100:.1f}%)")
        
        # Severity distribution
        print("\nSeverity Distribution:")
        severity_counts = quality_events_df['severity'].value_counts().sort_index()
        for severity, count in severity_counts.items():
            print(f"  Severity {severity}: {count} ({count/len(quality_events_df)*100:.1f}%)")
        
        # Status distribution
        print("\nStatus Distribution:")
        status_counts = quality_events_df['status'].value_counts()
        for status, count in status_counts.items():
            print(f"  {status}: {count} ({count/len(quality_events_df)*100:.1f}%)")
        
        # Root cause distribution (for closed events)
        closed_events = quality_events_df[quality_events_df['status'] == "Closed"]
        if len(closed_events) > 0:
            print("\nRoot Cause Distribution (Closed Events):")
            root_cause_counts = closed_events['root_cause'].value_counts().head(10)
            for cause, count in root_cause_counts.items():
                if cause:  # Skip empty causes
                    print(f"  {cause}: {count} ({count/len(closed_events)*100:.1f}%)")
        
        # Time to close
        if len(closed_events) > 0:
            # Convert to datetime
            closed_events['detection_date'] = pd.to_datetime(closed_events['detection_date'])
            closed_events['closure_date'] = pd.to_datetime(closed_events['closure_date'])
            
            # Calculate days to close
            closed_events['days_to_close'] = (closed_events['closure_date'] - closed_events['detection_date']).dt.days
            
            print("\nTime to Close Statistics:")
            print(f"  Average days to close: {closed_events['days_to_close'].mean():.1f} days")
            print(f"  Minimum days to close: {closed_events['days_to_close'].min()} days")
            print(f"  Maximum days to close: {closed_events['days_to_close'].max()} days")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    material_lots_df = load_material_lots_data()
    products_df = load_products_data()
    batches_df = load_batches_data()
    work_orders_df = load_work_orders_data()
    equipment_df = load_equipment_data()
    personnel_df = load_personnel_data()
    
    if material_lots_df is not None:
        # Generate quality tests data
        quality_tests_df = generate_quality_tests(
            material_lots_df,
            products_df,
            batches_df,
            work_orders_df,
            equipment_df,
            personnel_df,
            num_tests=500,  # Generate 500 quality test records
            output_file="data/quality_tests.csv"
        )
        
        # Generate quality events based on tests
        if quality_tests_df is not None:
            # Try to load process areas data if available
            try:
                process_areas_df = pd.read_csv("data/process_areas.csv")
            except FileNotFoundError:
                process_areas_df = None
            
            quality_events_df = generate_quality_events(
                quality_tests_df,
                material_lots_df,
                batches_df,
                equipment_df,
                process_areas_df,
                personnel_df,
                num_events=100,  # Generate 100 quality event records
                output_file="data/quality_events.csv"
            )
        else:
            quality_events_df = None
        
        # Display statistics
        if quality_tests_df is not None:
            display_statistics(quality_tests_df, quality_events_df)
            
            # Display sample data
            print("\nSample quality tests data (first 5 records):")
            print(quality_tests_df.head(5))
            
            if quality_events_df is not None:
                print("\nSample quality events data (first 5 records):")
                print(quality_events_df.head(5))

Note: Products data file data/products.csv not found.
Quality tests will be generated with synthetic product IDs.
Note: Personnel data file data/personnel.csv not found.
Quality tests will be generated with synthetic inspector IDs.
Generating synthetic product IDs...
Generating synthetic inspector IDs...
Successfully generated 500 quality test records.
Data saved to data/quality_tests.csv
Successfully generated 100 quality event records.
Data saved to data/quality_events.csv

Quality Tests Statistics:
Total quality tests: 500

Test Type Distribution:
  Chemical Analysis: 97 (19.4%)
  Visual Inspection: 83 (16.6%)
  Physical Test: 80 (16.0%)
  Functional Test: 47 (9.4%)
  Dimensional Check: 44 (8.8%)
  Microbiological: 42 (8.4%)
  Release Testing: 33 (6.6%)
  Stability Test: 28 (5.6%)
  Identification Test: 25 (5.0%)
  Impurity Test: 21 (4.2%)

Test Result Distribution:
  Pass: 474 (94.8%)
  Fail: 26 (5.2%)

Tests flagged for retest: 31 (6.2%)

Failed Tests by Type:
  Chemical Analysis:

/tmp/ipykernel_114075/3498726793.py:824: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_events['detection_date'] = pd.to_datetime(closed_events['detection_date'])
/tmp/ipykernel_114075/3498726793.py:825: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_events['closure_date'] = pd.to_datetime(closed_events['closure_date'])
/tmp/ipykernel_114075/3498726793.py:828: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Resource Utilization 

In [12]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Error: Equipment data file {equipment_file} not found.")
        print("Please run the equipment data generation script first.")
        return None

def load_personnel_data(personnel_file="data/personnel.csv"):
    """
    Load the previously generated personnel data if available
    
    Parameters:
    - personnel_file: CSV file containing personnel data
    
    Returns:
    - DataFrame containing the personnel data or None if not available
    """
    try:
        return pd.read_csv(personnel_file)
    except FileNotFoundError:
        print(f"Note: Personnel data file {personnel_file} not found.")
        print("Resource utilization will be generated with synthetic personnel IDs.")
        return None

def load_material_lots_data(material_lots_file="data/material_lots.csv"):
    """
    Load the previously generated material lots data if available
    
    Parameters:
    - material_lots_file: CSV file containing material lots data
    
    Returns:
    - DataFrame containing the material lots data or None if not available
    """
    try:
        return pd.read_csv(material_lots_file)
    except FileNotFoundError:
        print(f"Note: Material lots data file {material_lots_file} not found.")
        print("Resource utilization will be generated with synthetic material IDs.")
        return None

def load_work_orders_data(work_orders_file="data/work_orders.csv"):
    """
    Load the previously generated work orders data if available
    
    Parameters:
    - work_orders_file: CSV file containing work orders data
    
    Returns:
    - DataFrame containing the work orders data or None if not available
    """
    try:
        return pd.read_csv(work_orders_file)
    except FileNotFoundError:
        print(f"Note: Work orders data file {work_orders_file} not found.")
        print("Resource utilization will be generated with synthetic work order IDs.")
        return None

def generate_resource_utilization(equipment_df, personnel_df=None, material_lots_df=None, work_orders_df=None,
                               num_records=1000, time_interval_minutes=60, start_time=None, end_time=None,
                               output_file="data/resource_utilization.csv"):
    """
    Generate synthetic data for the ResourceUtilization table from ISA-95 Level 3.
    
    Parameters:
    - equipment_df: DataFrame containing equipment data
    - personnel_df: DataFrame containing personnel data (optional)
    - material_lots_df: DataFrame containing material lots data (optional)
    - work_orders_df: DataFrame containing work orders data (optional)
    - num_records: Number of resource utilization records to generate
    - time_interval_minutes: Time interval between records in minutes
    - start_time: Start time for utilization data (defaults to 30 days ago)
    - end_time: End time for utilization data (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing a sample of the generated resource utilization data
    """
    if equipment_df is None or len(equipment_df) == 0:
        print("Error: No equipment data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=30)
    if end_time is None:
        end_time = datetime.now()
    
    # Get work order IDs if available, otherwise generate synthetic ones
    if work_orders_df is not None and len(work_orders_df) > 0:
        work_order_ids = work_orders_df['work_order_id'].unique().tolist()
    else:
        print("Generating synthetic work order IDs...")
        work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(30)]
    
    # Create resource IDs from available data
    resource_ids = []
    resource_types = []
    
    # Add equipment resources
    for _, equipment in equipment_df.iterrows():
        resource_ids.append(equipment['equipment_id'])
        resource_types.append("Equipment")
    
    # Add personnel resources if available
    if personnel_df is not None and len(personnel_df) > 0:
        for _, person in personnel_df.iterrows():
            resource_ids.append(person['personnel_id'])
            resource_types.append("Personnel")
    else:
        # Add synthetic personnel
        for i in range(20):
            resource_ids.append(f"PERS-{uuid.uuid4().hex[:8].upper()}")
            resource_types.append("Personnel")
    
    # Add material resources if available
    if material_lots_df is not None and len(material_lots_df) > 0:
        # Only include active materials
        active_materials = material_lots_df[material_lots_df['status'] == 'Active']
        if len(active_materials) > 0:
            for _, material in active_materials.iterrows():
                resource_ids.append(material['lot_id'])
                resource_types.append("Material")
        else:
            # Add some materials even if none are active
            for _, material in material_lots_df.sample(min(20, len(material_lots_df))).iterrows():
                resource_ids.append(material['lot_id'])
                resource_types.append("Material")
    else:
        # Add synthetic materials
        for i in range(15):
            resource_ids.append(f"LOT-{uuid.uuid4().hex[:8].upper()}")
            resource_types.append("Material")
    
    # Add utility resources (synthetic)
    utility_types = ["Electricity", "Water", "Steam", "Compressed Air", "Cooling Water", "Natural Gas", "Nitrogen"]
    for utility in utility_types:
        resource_ids.append(f"UTIL-{utility.upper().replace(' ', '')}")
        resource_types.append("Utility")
    
    # Create a resource map for lookup
    resource_map = dict(zip(resource_ids, resource_types))
    
    # Define downtime reasons by resource type
    downtime_reasons = {
        "Equipment": [
            "Preventive Maintenance", "Breakdown", "Setup/Changeover", "Calibration", 
            "Cleaning", "Operator Break", "Material Shortage", "Quality Issue", 
            "Scheduled Maintenance", "Tool Change", "Software Update", "Power Outage"
        ],
        "Personnel": [
            "Break", "Training", "Meeting", "Shift Change", "Absence", 
            "Documentation", "Administrative Task", "Support Activity"
        ],
        "Material": [
            "Quality Hold", "Awaiting Test Results", "Inventory Count", 
            "Transfer in Progress", "Shortage", "Replenishment"
        ],
        "Utility": [
            "Maintenance", "Supply Interruption", "External Outage", 
            "Capacity Limit", "Pressure Drop", "Temperature Deviation"
        ]
    }
    
    # Generate timestamp sequence
    time_range_minutes = int((end_time - start_time).total_seconds() / 60)
    num_intervals = min(num_records, time_range_minutes // time_interval_minutes)
    
    timestamps = [
        start_time + timedelta(minutes=i * time_interval_minutes)
        for i in range(num_intervals)
    ]
    
    # Prepare to write data directly to CSV for memory efficiency
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = [
            'timestamp', 'resource_id', 'resource_type', 'order_id',
            'utilization_percentage', 'planned_utilization', 'actual_utilization',
            'availability_status', 'downtime', 'downtime_reason'
        ]
        writer = pd.DataFrame(columns=fieldnames)
        
        print(f"Generating resource utilization data...")
        records_generated = 0
        data_rows = []
        
        # For each timestamp, generate utilization for a subset of resources
        for timestamp in timestamps:
            # Determine how many resources to include at this timestamp
            # (not all resources are tracked at every interval)
            num_resources = random.randint(10, min(50, len(resource_ids)))
            
            # Select random resources to track at this interval
            selected_resources = random.sample(resource_ids, num_resources)
            
            # For each selected resource, generate utilization data
            for resource_id in selected_resources:
                resource_type = resource_map[resource_id]
                
                # Determine if associated with a work order
                if random.random() < 0.7:  # 70% associated with work order
                    order_id = random.choice(work_order_ids)
                else:
                    order_id = ""
                
                # Generate utilization data based on resource type
                if resource_type == "Equipment":
                    # Equipment tends to have higher utilization
                    planned_utilization = random.uniform(60, 95)
                    
                    # Determine if equipment is down
                    if random.random() < 0.1:  # 10% chance of downtime
                        availability_status = "Down"
                        actual_utilization = 0.0
                        downtime = time_interval_minutes
                        downtime_reason = random.choice(downtime_reasons["Equipment"])
                    else:
                        # Variation from planned (normally distributed)
                        variation = random.normalvariate(0, 10)  # Mean 0, std dev 10 percentage points
                        actual_utilization = max(0, min(100, planned_utilization + variation))
                        
                        if actual_utilization < 5:
                            availability_status = "Idle"
                            downtime = time_interval_minutes
                            downtime_reason = "No Production Scheduled"
                        else:
                            availability_status = "Running"
                            downtime = 0
                            downtime_reason = ""
                
                elif resource_type == "Personnel":
                    # Personnel utilization tends to be more varied
                    planned_utilization = random.uniform(50, 90)
                    
                    # Determine if personnel is unavailable
                    if random.random() < 0.15:  # 15% chance of unavailability
                        availability_status = "Unavailable"
                        actual_utilization = 0.0
                        downtime = time_interval_minutes
                        downtime_reason = random.choice(downtime_reasons["Personnel"])
                    else:
                        # Variation from planned (more variable than equipment)
                        variation = random.normalvariate(0, 15)  # Mean 0, std dev 15 percentage points
                        actual_utilization = max(0, min(100, planned_utilization + variation))
                        
                        if actual_utilization < 10:
                            availability_status = "Available"
                            downtime = time_interval_minutes
                            downtime_reason = "Waiting for Assignment"
                        else:
                            availability_status = "Assigned"
                            downtime = 0
                            downtime_reason = ""
                
                elif resource_type == "Material":
                    # Material utilization is typically lower and spiky
                    planned_utilization = random.uniform(20, 60)
                    
                    # Determine if material is unavailable
                    if random.random() < 0.05:  # 5% chance of unavailability
                        availability_status = "On Hold"
                        actual_utilization = 0.0
                        downtime = time_interval_minutes
                        downtime_reason = random.choice(downtime_reasons["Material"])
                    else:
                        # Materials often have bursts of usage
                        if random.random() < 0.3:  # 30% chance of high usage
                            actual_utilization = random.uniform(70, 100)
                            availability_status = "In Use"
                            downtime = 0
                            downtime_reason = ""
                        else:
                            actual_utilization = random.uniform(0, planned_utilization)
                            if actual_utilization < 5:
                                availability_status = "Available"
                                downtime = time_interval_minutes
                                downtime_reason = "Not Required"
                            else:
                                availability_status = "In Use"
                                downtime = 0
                                downtime_reason = ""
                
                else:  # Utility
                    # Utilities typically have high availability but variable usage
                    planned_utilization = random.uniform(30, 70)
                    
                    # Determine if utility is unavailable
                    if random.random() < 0.03:  # 3% chance of outage
                        availability_status = "Outage"
                        actual_utilization = 0.0
                        downtime = time_interval_minutes
                        downtime_reason = random.choice(downtime_reasons["Utility"])
                    else:
                        # Utilities can have peak usage periods
                        hour_of_day = timestamp.hour
                        
                        # Higher usage during standard working hours
                        if 8 <= hour_of_day <= 17:
                            usage_factor = random.uniform(0.8, 1.2)
                        else:
                            usage_factor = random.uniform(0.5, 0.9)
                            
                        actual_utilization = min(100, planned_utilization * usage_factor)
                        availability_status = "Available"
                        downtime = 0
                        downtime_reason = ""
                
                # Calculate utilization percentage (actual vs. planned)
                if planned_utilization > 0:
                    utilization_percentage = (actual_utilization / planned_utilization) * 100
                else:
                    utilization_percentage = 0.0
                
                # Round values for cleaner data
                planned_utilization = round(planned_utilization, 1)
                actual_utilization = round(actual_utilization, 1)
                utilization_percentage = round(utilization_percentage, 1)
                
                # Add row to data
                data_rows.append({
                    'timestamp': timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                    'resource_id': resource_id,
                    'resource_type': resource_type,
                    'order_id': order_id,
                    'utilization_percentage': utilization_percentage,
                    'planned_utilization': planned_utilization,
                    'actual_utilization': actual_utilization,
                    'availability_status': availability_status,
                    'downtime': downtime,
                    'downtime_reason': downtime_reason
                })
                
                records_generated += 1
                if records_generated % 10000 == 0:
                    print(f"Generated {records_generated} utilization records so far...")
                
                # If we've hit our target number of records, stop
                if records_generated >= num_records:
                    break
            
            # If we've hit our target number of records, stop
            if records_generated >= num_records:
                break
        
        # Create DataFrame from rows and save to CSV
        writer = pd.DataFrame(data_rows)
        writer.to_csv(output_file, index=False)
    
    print(f"Successfully generated {records_generated} resource utilization records.")
    print(f"Data saved to {output_file}")
    
    # Return a sample of the data (first 1000 rows) for preview
    return pd.read_csv(output_file, nrows=1000)

def display_statistics(resource_utilization_df):
    """
    Display basic statistics about the generated resource utilization data
    
    Parameters:
    - resource_utilization_df: DataFrame containing resource utilization data
    """
    if resource_utilization_df is None or len(resource_utilization_df) == 0:
        print("No resource utilization data to analyze.")
        return
    
    print("\nResource Utilization Statistics:")
    print(f"Total utilization records: {len(resource_utilization_df)}")
    
    # Resource type distribution
    print("\nResource Type Distribution:")
    type_counts = resource_utilization_df['resource_type'].value_counts()
    for res_type, count in type_counts.items():
        print(f"  {res_type}: {count} ({count/len(resource_utilization_df)*100:.1f}%)")
    
    # Availability status distribution
    print("\nAvailability Status Distribution:")
    status_counts = resource_utilization_df['availability_status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count} ({count/len(resource_utilization_df)*100:.1f}%)")
    
    # Downtime analysis
    downtime_records = resource_utilization_df[resource_utilization_df['downtime'] > 0]
    downtime_pct = len(downtime_records) / len(resource_utilization_df) * 100
    print(f"\nRecords with downtime: {len(downtime_records)} ({downtime_pct:.1f}%)")
    
    if len(downtime_records) > 0:
        print("\nDowntime Reason Distribution:")
        reason_counts = downtime_records['downtime_reason'].value_counts().head(10)
        for reason, count in reason_counts.items():
            print(f"  {reason}: {count} ({count/len(downtime_records)*100:.1f}%)")
    
    # Utilization statistics by resource type
    print("\nUtilization Statistics by Resource Type:")
    for res_type in type_counts.index:
        type_data = resource_utilization_df[resource_utilization_df['resource_type'] == res_type]
        
        print(f"\n  {res_type}:")
        print(f"    Records: {len(type_data)}")
        print(f"    Avg Planned Utilization: {type_data['planned_utilization'].mean():.1f}%")
        print(f"    Avg Actual Utilization: {type_data['actual_utilization'].mean():.1f}%")
        print(f"    Avg Utilization Percentage: {type_data['utilization_percentage'].mean():.1f}%")
        
        # Downtime specific to this resource type
        type_downtime = type_data[type_data['downtime'] > 0]
        if len(type_downtime) > 0:
            downtime_pct = len(type_downtime) / len(type_data) * 100
            print(f"    Downtime Percentage: {downtime_pct:.1f}%")
            print(f"    Top Downtime Reason: {type_downtime['downtime_reason'].value_counts().index[0]}")
    
    # Time-based analysis
    resource_utilization_df['timestamp'] = pd.to_datetime(resource_utilization_df['timestamp'])
    
    # Group by day
    resource_utilization_df['day'] = resource_utilization_df['timestamp'].dt.date
    daily_utilization = resource_utilization_df.groupby('day')['actual_utilization'].mean()
    
    print("\nDaily Average Utilization:")
    for day, util in daily_utilization.head(7).items():
        print(f"  {day}: {util:.1f}%")
    
    # Work order connections
    wo_count = resource_utilization_df['order_id'].apply(lambda x: x != "").sum()
    wo_pct = wo_count / len(resource_utilization_df) * 100
    print(f"\nRecords associated with work orders: {wo_count} ({wo_pct:.1f}%)")
    
    # Resource count
    resource_count = resource_utilization_df['resource_id'].nunique()
    print(f"\nTotal unique resources tracked: {resource_count}")
    
    # Resources by type
    print("Unique resources by type:")
    for res_type in type_counts.index:
        type_data = resource_utilization_df[resource_utilization_df['resource_type'] == res_type]
        type_resources = type_data['resource_id'].nunique()
        print(f"  {res_type}: {type_resources}")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    equipment_df = load_equipment_data()
    personnel_df = load_personnel_data()
    material_lots_df = load_material_lots_data()
    work_orders_df = load_work_orders_data()
    
    if equipment_df is not None:
        # Generate resource utilization data
        resource_utilization_df = generate_resource_utilization(
            equipment_df,
            personnel_df,
            material_lots_df,
            work_orders_df,
            num_records=1000,  # Generate 1000 utilization records
            time_interval_minutes=60,  # Hourly intervals
            output_file="data/resource_utilization.csv"
        )
        
        # Display statistics
        if resource_utilization_df is not None:
            display_statistics(resource_utilization_df)
            
            # Display sample data
            print("\nSample resource utilization data (first 5 records):")
            print(resource_utilization_df.head(5))

Note: Personnel data file data/personnel.csv not found.
Resource utilization will be generated with synthetic personnel IDs.
Generating resource utilization data...
Successfully generated 1000 resource utilization records.
Data saved to data/resource_utilization.csv

Resource Utilization Statistics:
Total utilization records: 1000

Resource Type Distribution:
  Material: 521 (52.1%)
  Equipment: 416 (41.6%)
  Personnel: 50 (5.0%)
  Utility: 13 (1.3%)

Availability Status Distribution:
  In Use: 433 (43.3%)
  Running: 372 (37.2%)
  Available: 70 (7.0%)
  Down: 44 (4.4%)
  Assigned: 43 (4.3%)
  On Hold: 31 (3.1%)
  Unavailable: 7 (0.7%)

Records with downtime: 139 (13.9%)

Downtime Reason Distribution:
  Not Required: 57 (41.0%)
  Replenishment: 8 (5.8%)
  Operator Break: 7 (5.0%)
  Quality Hold: 6 (4.3%)
  Inventory Count: 5 (3.6%)
  Scheduled Maintenance: 5 (3.6%)
  Cleaning: 5 (3.6%)
  Transfer in Progress: 5 (3.6%)
  Power Outage: 4 (2.9%)
  Tool Change: 4 (2.9%)

Utilization Statist

Maintenance Activities

In [ ]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Error: Equipment data file {equipment_file} not found.")
        print("Please run the equipment data generation script first.")
        return None

def load_work_orders_data(work_orders_file="data/work_orders.csv"):
    """
    Load the previously generated work orders data if available
    
    Parameters:
    - work_orders_file: CSV file containing work orders data
    
    Returns:
    - DataFrame containing the work orders data or None if not available
    """
    try:
        return pd.read_csv(work_orders_file)
    except FileNotFoundError:
        print(f"Note: Work orders data file {work_orders_file} not found.")
        print("Maintenance activities will be generated with synthetic work order IDs.")
        return None

def load_personnel_data(personnel_file="data/personnel.csv"):
    """
    Load the previously generated personnel data if available
    
    Parameters:
    - personnel_file: CSV file containing personnel data
    
    Returns:
    - DataFrame containing the personnel data or None if not available
    """
    try:
        return pd.read_csv(personnel_file)
    except FileNotFoundError:
        print(f"Note: Personnel data file {personnel_file} not found.")
        print("Maintenance activities will be generated with synthetic technician IDs.")
        return None

def generate_maintenance_activities(equipment_df, work_orders_df=None, personnel_df=None,
                                  num_activities=300, start_time=None, end_time=None,
                                  output_file="data/maintenance_activities.csv"):
    """
    Generate synthetic data for the MaintenanceActivities table from ISA-95 Level 3.
    
    Parameters:
    - equipment_df: DataFrame containing equipment data
    - work_orders_df: DataFrame containing work orders data (optional)
    - personnel_df: DataFrame containing personnel data (optional)
    - num_activities: Number of maintenance activity records to generate
    - start_time: Start time for activity dates (defaults to 365 days ago)
    - end_time: End time for activity dates (defaults to 30 days in the future)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated maintenance activities data
    """
    if equipment_df is None or len(equipment_df) == 0:
        print("Error: No equipment data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=365)
    if end_time is None:
        end_time = datetime.now() + timedelta(days=30)
    
    # Extract or generate work order IDs
    if work_orders_df is not None and len(work_orders_df) > 0:
        # Filter for maintenance work orders
        maintenance_wos = work_orders_df[work_orders_df['work_order_type'] == 'Maintenance']
        
        if len(maintenance_wos) > 0:
            work_order_ids = maintenance_wos['work_order_id'].tolist()
        else:
            # If no maintenance work orders, generate synthetic ones
            print("No maintenance work orders found. Generating synthetic work order IDs...")
            work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_activities)]
    else:
        # Generate synthetic work order IDs
        print("Generating synthetic work order IDs...")
        work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_activities)]
    
    # Extract or generate technician IDs
    if personnel_df is not None and len(personnel_df) > 0:
        technician_ids = personnel_df['personnel_id'].sample(min(20, len(personnel_df))).tolist()
    else:
        # Generate synthetic technician IDs
        print("Generating synthetic technician IDs...")
        technician_ids = [f"TECH-{uuid.uuid4().hex[:8].upper()}" for _ in range(15)]
    
    # Define maintenance activity types and their probabilities
    activity_types = {
        "Preventive": 0.4,       # Regular scheduled maintenance
        "Corrective": 0.3,       # Fix after failure
        "Predictive": 0.1,       # Based on condition monitoring
        "Inspection": 0.1,       # Regular checks
        "Calibration": 0.05,     # Calibrating instruments
        "Overhaul": 0.03,        # Major maintenance
        "Upgrade": 0.02          # Improving equipment
    }
    
    # Define priority levels
    priority_levels = [1, 2, 3, 4, 5]  # 1 = highest, 5 = lowest
    priority_weights = [0.1, 0.2, 0.4, 0.2, 0.1]  # Most activities are medium priority
    
    # Define maintenance activity durations (in hours) by type
    activity_durations = {
        "Preventive": (2, 8),       # 2-8 hours
        "Corrective": (1, 24),      # 1-24 hours
        "Predictive": (1, 4),       # 1-4 hours
        "Inspection": (0.5, 2),     # 30 min - 2 hours
        "Calibration": (1, 6),      # 1-6 hours
        "Overhaul": (8, 72),        # 8-72 hours
        "Upgrade": (4, 48)          # 4-48 hours
    }
    
    # Define activity statuses and their time-based probabilities
    activity_statuses = ["Planned", "Scheduled", "In Progress", "Completed", "Canceled"]
    
    # Define common descriptions by activity type
    activity_descriptions = {
        "Preventive": [
            "Routine maintenance per schedule",
            "Preventive maintenance as per manual",
            "Scheduled lubrication and inspection",
            "Regular service check",
            "Planned component replacement"
        ],
        "Corrective": [
            "Repair after failure",
            "Fix mechanical issue",
            "Replace worn component",
            "Repair electrical fault",
            "Emergency fix after breakdown"
        ],
        "Predictive": [
            "Maintenance based on vibration analysis",
            "Service based on oil analysis results",
            "Pre-emptive repair based on monitoring",
            "Condition-based maintenance",
            "Thermography-indicated maintenance"
        ],
        "Inspection": [
            "Safety inspection",
            "Regulatory compliance check",
            "Visual inspection of components",
            "Operational check",
            "Performance verification"
        ],
        "Calibration": [
            "Sensor calibration",
            "Instrument accuracy verification",
            "Scale calibration",
            "Control system tuning",
            "Measurement system adjustment"
        ],
        "Overhaul": [
            "Complete system teardown and rebuild",
            "Major component replacement",
            "Full mechanical overhaul",
            "Comprehensive service",
            "Complete system restoration"
        ],
        "Upgrade": [
            "Software update installation",
            "Hardware upgrade",
            "Performance enhancement modification",
            "Component upgrade installation",
            "Feature addition"
        ]
    }
    
    # Generate data structure
    data = {
        "activity_id": [f"MAINT-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_activities)],
        "activity_type": [],
        "equipment_id": [],
        "work_order_id": [],
        "planned_start_date": [],
        "actual_start_date": [],
        "planned_end_date": [],
        "actual_end_date": [],
        "status": [],
        "priority": [],
        "description": [],
        "technician_id": [],
        "downtime_required": [],
        "actual_downtime_minutes": []
    }
    
    # Create date distribution for maintenance activities
    # More activities in recent past and near future, fewer in distant past/future
    date_weights = []
    time_range_days = (end_time - start_time).days
    
    for i in range(time_range_days):
        # Weight activities to be more common in recent times
        days_from_now = abs((start_time + timedelta(days=i) - datetime.now()).days)
        if days_from_now <= 30:
            # Recent past or near future (high density)
            weight = 1.0
        elif days_from_now <= 90:
            # Medium past/future (medium density)
            weight = 0.5
        else:
            # Distant past/future (low density)
            weight = 0.2
        date_weights.append(weight)
    
    # Normalize weights
    total_weight = sum(date_weights)
    date_weights = [w / total_weight for w in date_weights]
    
    # Generate data for each maintenance activity
    for i in range(num_activities):
        # Select activity type (weighted random)
        activity_type = random.choices(
            list(activity_types.keys()), 
            weights=list(activity_types.values())
        )[0]
        data["activity_type"].append(activity_type)
        
        # Select equipment ID (favor older equipment for more maintenance)
        if 'installation_date' in equipment_df.columns:
            # Convert to datetime if it's not already
            if not pd.api.types.is_datetime64_dtype(equipment_df['installation_date']):
                equipment_df['installation_date'] = pd.to_datetime(equipment_df['installation_date'], errors='coerce')
            
            # Calculate equipment age
            current_date = datetime.now()
            equipment_df['age_days'] = (current_date - equipment_df['installation_date']).dt.days
            
            # Weight by age (older equipment needs more maintenance)
            weights = equipment_df['age_days'].fillna(365).values
            weights = weights / max(1, weights.sum())  # Normalize
            
            # Select equipment with probability proportional to age
            selected_idx = random.choices(range(len(equipment_df)), weights=weights)[0]
            equipment_id = equipment_df.iloc[selected_idx]['equipment_id']
        else:
            # If no installation date, select randomly
            equipment_id = random.choice(equipment_df['equipment_id'].tolist())
        
        data["equipment_id"].append(equipment_id)
        
        # Select work order ID
        if len(work_order_ids) > 0:
            data["work_order_id"].append(random.choice(work_order_ids))
        else:
            data["work_order_id"].append(f"WO-{uuid.uuid4().hex[:8].upper()}")
        
        # Generate planned start date
        day_idx = random.choices(range(time_range_days), weights=date_weights)[0]
        planned_start_date = start_time + timedelta(days=day_idx)
        
        # Add random hours to make times more realistic
        planned_start_date += timedelta(hours=random.randint(7, 16))  # Business hours
        
        data["planned_start_date"].append(planned_start_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        # Get duration range for this activity type
        min_hours, max_hours = activity_durations[activity_type]
        
        # Generate planned duration
        planned_duration_hours = random.uniform(min_hours, max_hours)
        planned_end_date = planned_start_date + timedelta(hours=planned_duration_hours)
        
        data["planned_end_date"].append(planned_end_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        # Determine status based on dates
        current_date = datetime.now()
        
        if planned_start_date > current_date:
            # Future activity
            if (planned_start_date - current_date).days < 7:
                # Near future
                status = random.choices(["Planned", "Scheduled"], weights=[0.3, 0.7])[0]
            else:
                # More distant future
                status = "Planned"
            
            # Future activities don't have actual dates yet
            data["actual_start_date"].append("")
            data["actual_end_date"].append("")
            data["actual_downtime_minutes"].append("")
            
        elif planned_start_date <= current_date and planned_end_date > current_date:
            # Current activity
            if random.random() < 0.8:  # 80% chance it started on time
                status = "In Progress"
                
                # Activity started but not finished
                actual_start_date = planned_start_date + timedelta(minutes=random.randint(-60, 60))
                data["actual_start_date"].append(actual_start_date.strftime("%Y-%m-%d %H:%M:%S"))
                data["actual_end_date"].append("")
                
                # Partial downtime so far
                current_downtime = (current_date - actual_start_date).total_seconds() / 60
                data["actual_downtime_minutes"].append(round(current_downtime))
                
            else:
                # Activity delayed
                status = random.choices(["Planned", "Scheduled"], weights=[0.3, 0.7])[0]
                data["actual_start_date"].append("")
                data["actual_end_date"].append("")
                data["actual_downtime_minutes"].append("")
                
        else:
            # Past activity
            if random.random() < 0.9:  # 90% chance it was completed
                status = "Completed"
                
                # Actual start date might vary from planned
                start_variation_minutes = random.randint(-120, 120)  # +/- 2 hours
                actual_start_date = planned_start_date + timedelta(minutes=start_variation_minutes)
                
                # Actual duration might vary from planned
                duration_variation = random.normalvariate(1.0, 0.2)  # Mean 1.0, std dev 0.2
                actual_duration_hours = max(0.1, planned_duration_hours * duration_variation)
                actual_end_date = actual_start_date + timedelta(hours=actual_duration_hours)
                
                data["actual_start_date"].append(actual_start_date.strftime("%Y-%m-%d %H:%M:%S"))
                data["actual_end_date"].append(actual_end_date.strftime("%Y-%m-%d %H:%M:%S"))
                
                # Calculate actual downtime
                actual_downtime = actual_duration_hours * 60  # Convert to minutes
                data["actual_downtime_minutes"].append(round(actual_downtime))
                
            else:
                # Activity was canceled
                status = "Canceled"
                data["actual_start_date"].append("")
                data["actual_end_date"].append("")
                data["actual_downtime_minutes"].append("")
        
        data["status"].append(status)
        
        # Set priority (weighted random)
        priority = random.choices(priority_levels, weights=priority_weights)[0]
        
        # For corrective maintenance, increase priority (more urgent)
        if activity_type == "Corrective" and priority > 2:
            priority -= 1
            
        data["priority"].append(priority)
        
        # Set description
        if activity_type in activity_descriptions:
            description = random.choice(activity_descriptions[activity_type])
        else:
            description = f"{activity_type} maintenance activity"
            
        data["description"].append(description)
        
        # Assign technician
        data["technician_id"].append(random.choice(technician_ids))
        
        # Determine if downtime is required
        # Certain activity types almost always require downtime
        if activity_type in ["Corrective", "Overhaul", "Upgrade"]:
            downtime_required = random.random() < 0.95  # 95% require downtime
        elif activity_type in ["Preventive", "Calibration"]:
            downtime_required = random.random() < 0.7  # 70% require downtime
        else:
            downtime_required = random.random() < 0.3  # 30% require downtime
            
        data["downtime_required"].append(downtime_required)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} maintenance activity records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(maintenance_activities_df):
    """
    Display basic statistics about the generated maintenance activities data
    
    Parameters:
    - maintenance_activities_df: DataFrame containing maintenance activities data
    """
    if maintenance_activities_df is None or len(maintenance_activities_df) == 0:
        print("No maintenance activities data to analyze.")
        return
    
    print("\nMaintenance Activities Statistics:")
    print(f"Total maintenance activities: {len(maintenance_activities_df)}")
    
    # Activity type distribution
    print("\nActivity Type Distribution:")
    type_counts = maintenance_activities_df['activity_type'].value_counts()
    for activity_type, count in type_counts.items():
        print(f"  {activity_type}: {count} ({count/len(maintenance_activities_df)*100:.1f}%)")
    
    # Status distribution
    print("\nStatus Distribution:")
    status_counts = maintenance_activities_df['status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count} ({count/len(maintenance_activities_df)*100:.1f}%)")
    
    # Priority distribution
    print("\nPriority Distribution:")
    priority_counts = maintenance_activities_df['priority'].value_counts().sort_index()
    for priority, count in priority_counts.items():
        print(f"  Priority {priority}: {count} ({count/len(maintenance_activities_df)*100:.1f}%)")
    
    # Downtime required
    downtime_required = maintenance_activities_df['downtime_required'].sum()
    print(f"\nActivities requiring downtime: {downtime_required} ({downtime_required/len(maintenance_activities_df)*100:.1f}%)")
    
    # Convert date columns to datetime for analysis
    maintenance_activities_df['planned_start_date'] = pd.to_datetime(maintenance_activities_df['planned_start_date'], errors='coerce')
    maintenance_activities_df['planned_end_date'] = pd.to_datetime(maintenance_activities_df['planned_end_date'], errors='coerce')
    
    # Filter out empty strings before conversion
    maintenance_activities_df['actual_start_date'] = pd.to_datetime(
        maintenance_activities_df['actual_start_date'].replace('', pd.NaT), errors='coerce'
    )
    maintenance_activities_df['actual_end_date'] = pd.to_datetime(
        maintenance_activities_df['actual_end_date'].replace('', pd.NaT), errors='coerce'
    )
    
    # Calculate planned duration
    maintenance_activities_df['planned_duration_hours'] = (
        maintenance_activities_df['planned_end_date'] - maintenance_activities_df['planned_start_date']
    ).dt.total_seconds() / 3600
    
    print("\nPlanned Duration Statistics:")
    print(f"  Average planned duration: {maintenance_activities_df['planned_duration_hours'].mean():.1f} hours")
    print(f"  Minimum planned duration: {maintenance_activities_df['planned_duration_hours'].min():.1f} hours")
    print(f"  Maximum planned duration: {maintenance_activities_df['planned_duration_hours'].max():.1f} hours")
    
    # Calculate actual duration for completed activities
    completed = maintenance_activities_df[maintenance_activities_df['status'] == "Completed"]
    
    if len(completed) > 0:
        completed['actual_duration_hours'] = (
            completed['actual_end_date'] - completed['actual_start_date']
        ).dt.total_seconds() / 3600
        
        print("\nActual Duration Statistics (Completed Activities):")
        print(f"  Average actual duration: {completed['actual_duration_hours'].mean():.1f} hours")
        print(f"  Minimum actual duration: {completed['actual_duration_hours'].min():.1f} hours")
        print(f"  Maximum actual duration: {completed['actual_duration_hours'].max():.1f} hours")
        
        # Calculate duration variance
        completed['duration_variance'] = completed['actual_duration_hours'] - completed['planned_duration_hours']
        completed['duration_variance_pct'] = (completed['duration_variance'] / completed['planned_duration_hours']) * 100
        
        print("\nDuration Variance (Completed Activities):")
        print(f"  Average variance: {completed['duration_variance'].mean():.1f} hours")
        print(f"  Average variance percentage: {completed['duration_variance_pct'].mean():.1f}%")
        
        # Categorize activities by variance
        on_time = len(completed[abs(completed['duration_variance_pct']) <= 10])
        faster = len(completed[completed['duration_variance_pct'] < -10])
        slower = len(completed[completed['duration_variance_pct'] > 10])
        
        print(f"  On time (±10%): {on_time} ({on_time/len(completed)*100:.1f}%)")
        print(f"  Faster than planned: {faster} ({faster/len(completed)*100:.1f}%)")
        print(f"  Slower than planned: {slower} ({slower/len(completed)*100:.1f}%)")
    
    # Time distribution
    current_date = datetime.now()
    
    past_activities = maintenance_activities_df[maintenance_activities_df['planned_start_date'] < current_date]
    future_activities = maintenance_activities_df[maintenance_activities_df['planned_start_date'] >= current_date]
    
    print("\nTime Distribution:")
    print(f"  Past activities: {len(past_activities)} ({len(past_activities)/len(maintenance_activities_df)*100:.1f}%)")
    print(f"  Future activities: {len(future_activities)} ({len(future_activities)/len(maintenance_activities_df)*100:.1f}%)")
    
    # Monthly distribution
    maintenance_activities_df['month'] = maintenance_activities_df['planned_start_date'].dt.to_period('M')
    monthly_counts = maintenance_activities_df['month'].value_counts().sort_index()
    
    print("\nMonthly Distribution (top 6 months):")
    for month, count in monthly_counts.head(6).items():
        print(f"  {month}: {count} activities")
    
    # Equipment distribution
    equipment_counts = maintenance_activities_df['equipment_id'].value_counts()
    
    print("\nTop 5 Equipment by Maintenance Frequency:")
    for equipment_id, count in equipment_counts.head(5).items():
        print(f"  {equipment_id}: {count} activities")
    
    # Technician distribution
    technician_counts = maintenance_activities_df['technician_id'].value_counts()
    
    print("\nTop 5 Technicians by Activity Assignment:")
    for technician_id, count in technician_counts.head(5).items():
        print(f"  {technician_id}: {count} activities")
    
    # Activity type by downtime requirement
    print("\nDowntime Requirement by Activity Type:")
    for activity_type in type_counts.index:
        type_data = maintenance_activities_df[maintenance_activities_df['activity_type'] == activity_type]
        downtime_count = type_data['downtime_required'].sum()
        print(f"  {activity_type}: {downtime_count}/{len(type_data)} ({downtime_count/len(type_data)*100:.1f}%)")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    equipment_df = load_equipment_data()
    work_orders_df = load_work_orders_data()
    personnel_df = load_personnel_data()
    
    if equipment_df is not None:
        # Generate maintenance activities data
        maintenance_activities_df = generate_maintenance_activities(
            equipment_df,
            work_orders_df,
            personnel_df,
            num_activities=300,  # Generate 300 maintenance activity records
            output_file="data/maintenance_activities.csv"
        )
        
        # Display statistics
        if maintenance_activities_df is not None:
            display_statistics(maintenance_activities_df)
            
            # Display sample data
            print("\nSample maintenance activities data (first 5 records):")
            print(maintenance_activities_df.head(5))

Note: Personnel data file data/personnel.csv not found.
Maintenance activities will be generated with synthetic technician IDs.
Generating synthetic technician IDs...
Successfully generated 300 maintenance activity records.
Data saved to data/maintenance_activities.csv

Maintenance Activities Statistics:
Total maintenance activities: 300

Activity Type Distribution:
  Preventive: 127 (42.3%)
  Corrective: 91 (30.3%)
  Inspection: 33 (11.0%)
  Predictive: 33 (11.0%)
  Calibration: 11 (3.7%)
  Overhaul: 4 (1.3%)
  Upgrade: 1 (0.3%)

Status Distribution:
  Completed: 220 (73.3%)
  Planned: 50 (16.7%)
  Canceled: 18 (6.0%)
  Scheduled: 12 (4.0%)

Priority Distribution:
  Priority 1: 28 (9.3%)
  Priority 2: 91 (30.3%)
  Priority 3: 115 (38.3%)
  Priority 4: 46 (15.3%)
  Priority 5: 20 (6.7%)

Activities requiring downtime: 197 (65.7%)

Planned Duration Statistics:
  Average planned duration: 7.4 hours
  Minimum planned duration: 0.5 hours
  Maximum planned duration: 66.9 hours

Actual Durat

/tmp/ipykernel_114075/861228857.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed['actual_duration_hours'] = (
/tmp/ipykernel_114075/861228857.py:475: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed['duration_variance'] = completed['actual_duration_hours'] - completed['planned_duration_hours']
/tmp/ipykernel_114075/861228857.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

Production Performance

In [14]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random
import os

def load_equipment_data(equipment_file="data/equipment.csv"):
    """
    Load the previously generated equipment data
    
    Parameters:
    - equipment_file: CSV file containing equipment data
    
    Returns:
    - DataFrame containing the equipment data
    """
    try:
        return pd.read_csv(equipment_file)
    except FileNotFoundError:
        print(f"Error: Equipment data file {equipment_file} not found.")
        print("Please run the equipment data generation script first.")
        return None

def load_work_orders_data(work_orders_file="data/work_orders.csv"):
    """
    Load the previously generated work orders data if available
    
    Parameters:
    - work_orders_file: CSV file containing work orders data
    
    Returns:
    - DataFrame containing the work orders data or None if not available
    """
    try:
        return pd.read_csv(work_orders_file)
    except FileNotFoundError:
        print(f"Note: Work orders data file {work_orders_file} not found.")
        print("Production performance will be generated with synthetic work order IDs.")
        return None

def load_maintenance_activities(maintenance_file="data/maintenance_activities.csv"):
    """
    Load the previously generated maintenance activities data if available
    
    Parameters:
    - maintenance_file: CSV file containing maintenance activities data
    
    Returns:
    - DataFrame containing the maintenance activities data or None if not available
    """
    try:
        return pd.read_csv(maintenance_file)
    except FileNotFoundError:
        print(f"Note: Maintenance activities file {maintenance_file} not found.")
        print("Production performance will not incorporate maintenance-related downtime.")
        return None

def load_equipment_states(equipment_states_file="data/equipment_states.csv"):
    """
    Load the previously generated equipment states data if available
    
    Parameters:
    - equipment_states_file: CSV file containing equipment states data
    
    Returns:
    - DataFrame containing the equipment states data or None if not available
    """
    try:
        return pd.read_csv(equipment_states_file)
    except FileNotFoundError:
        print(f"Note: Equipment states file {equipment_states_file} not found.")
        print("Production performance will not incorporate equipment state history.")
        return None

def generate_production_performance(equipment_df, work_orders_df=None, maintenance_df=None, equipment_states_df=None,
                                  num_periods=1000, time_period="Hour", 
                                  start_time=None, end_time=None,
                                  output_file="data/production_performance.csv"):
    """
    Generate synthetic data for the ProductionPerformance table from ISA-95 Level 3.
    
    Parameters:
    - equipment_df: DataFrame containing equipment data
    - work_orders_df: DataFrame containing work orders data (optional)
    - maintenance_df: DataFrame containing maintenance activities data (optional)
    - equipment_states_df: DataFrame containing equipment states data (optional)
    - num_periods: Number of performance records to generate
    - time_period: Time period for metrics (Hour, Shift, Day, Week)
    - start_time: Start time for performance data (defaults to 90 days ago)
    - end_time: End time for performance data (defaults to now)
    - output_file: CSV file to save the data
    
    Returns:
    - DataFrame containing the generated production performance data
    """
    if equipment_df is None or len(equipment_df) == 0:
        print("Error: No equipment data available.")
        return None
    
    # Set default time range if not provided
    if start_time is None:
        start_time = datetime.now() - timedelta(days=90)
    if end_time is None:
        end_time = datetime.now()
    
    # Determine time period duration in minutes
    if time_period == "Hour":
        period_minutes = 60
    elif time_period == "Shift":
        period_minutes = 480  # 8-hour shift
    elif time_period == "Day":
        period_minutes = 1440  # 24 hours
    elif time_period == "Week":
        period_minutes = 10080  # 7 days
    else:
        # Default to hourly
        period_minutes = 60
        time_period = "Hour"
    
    # Get work order IDs if available, otherwise generate synthetic ones
    if work_orders_df is not None and len(work_orders_df) > 0:
        # Filter for production work orders
        production_wos = work_orders_df[work_orders_df['work_order_type'] == 'Production']
        
        if len(production_wos) > 0:
            work_order_ids = production_wos['work_order_id'].tolist()
        else:
            # If no production work orders, generate synthetic ones
            print("No production work orders found. Generating synthetic work order IDs...")
            work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(50)]
    else:
        # Generate synthetic work order IDs
        print("Generating synthetic work order IDs...")
        work_order_ids = [f"WO-{uuid.uuid4().hex[:8].upper()}" for _ in range(50)]
    
    # Create shifts for time periods (used to maintain consistent shift IDs)
    if time_period == "Shift":
        # Create 3 shifts per day
        shift_ids = [f"SHIFT-{uuid.uuid4().hex[:8].upper()}" for _ in range(3)]
    else:
        # For non-shift time periods, create a few generic shift IDs
        shift_ids = [f"SHIFT-{uuid.uuid4().hex[:8].upper()}" for _ in range(4)]
    
    # Generate data structure
    data = {
        "performance_id": [f"PERF-{uuid.uuid4().hex[:8].upper()}" for _ in range(num_periods)],
        "equipment_id": [],
        "work_order_id": [],
        "shift_id": [],
        "timestamp": [],
        "time_period": [],
        "availability_percent": [],
        "performance_percent": [],
        "quality_percent": [],
        "oee_percent": [],
        "production_count": [],
        "reject_count": [],
        "downtime_minutes": [],
        "cycle_time_seconds": []
    }
    
    # Calculate time points for performance records
    time_range_minutes = int((end_time - start_time).total_seconds() / 60)
    
    # Determine how many time periods we can fit in the range
    max_periods = time_range_minutes // period_minutes
    
    if max_periods < num_periods:
        print(f"Warning: Time range can only fit {max_periods} {time_period} periods.")
        print(f"Reducing requested number of periods from {num_periods} to {max_periods}.")
        num_periods = max_periods
    
    # Select random time points within the range
    period_starts = []
    
    # If using hour or shift, ensure periods don't overlap
    if time_period in ["Hour", "Shift"]:
        # Create evenly spaced periods
        for i in range(num_periods):
            if i < max_periods:
                # For periods that fit within the range, space them evenly
                period_start = start_time + timedelta(minutes=i * period_minutes)
                period_starts.append(period_start)
            else:
                # If we need more periods than the range allows, start reusing time points
                # with different equipment
                period_start = period_starts[i % max_periods]
                period_starts.append(period_start)
    else:
        # For day and week, allow some overlap (different equipment can have records for the same day)
        for _ in range(num_periods):
            random_minutes = random.randint(0, time_range_minutes - period_minutes)
            period_start = start_time + timedelta(minutes=random_minutes)
            period_starts.append(period_start)
    
    # Define equipment categories for different performance profiles
    # Extract equipment types if available, otherwise create synthetic categories
    if 'equipment_type' in equipment_df.columns:
        equipment_categories = equipment_df['equipment_type'].unique().tolist()
    else:
        equipment_categories = [
            "Production", "Packaging", "Assembly", "Processing", 
            "Machining", "Filling", "Testing", "Utility"
        ]
    
    # Create performance profiles by equipment category
    performance_profiles = {}
    
    for category in equipment_categories:
        # Base performance parameters
        if category in ["Production", "Processing"]:
            # Process equipment typically has high availability, variable performance
            profile = {
                "availability": {"base": 0.92, "std": 0.05},
                "performance": {"base": 0.85, "std": 0.08},
                "quality": {"base": 0.98, "std": 0.02},
                "cycle_time": {"base": 120, "std": 20}
            }
        elif category in ["Packaging", "Filling"]:
            # Packaging equipment has moderate availability, high performance
            profile = {
                "availability": {"base": 0.88, "std": 0.07},
                "performance": {"base": 0.9, "std": 0.05},
                "quality": {"base": 0.99, "std": 0.01},
                "cycle_time": {"base": 30, "std": 5}
            }
        elif category in ["Assembly", "Machining"]:
            # Assembly and machining have lower availability, high quality
            profile = {
                "availability": {"base": 0.85, "std": 0.08},
                "performance": {"base": 0.8, "std": 0.1},
                "quality": {"base": 0.995, "std": 0.005},
                "cycle_time": {"base": 180, "std": 30}
            }
        elif category == "Testing":
            # Testing equipment has high availability, consistent performance
            profile = {
                "availability": {"base": 0.95, "std": 0.03},
                "performance": {"base": 0.9, "std": 0.04},
                "quality": {"base": 0.999, "std": 0.001},
                "cycle_time": {"base": 60, "std": 10}
            }
        else:
            # Default profile
            profile = {
                "availability": {"base": 0.9, "std": 0.06},
                "performance": {"base": 0.85, "std": 0.07},
                "quality": {"base": 0.98, "std": 0.02},
                "cycle_time": {"base": 90, "std": 15}
            }
        
        performance_profiles[category] = profile
    
    # Default production rate by category (units per hour)
    production_rates = {
        "Production": {"base": 100, "std": 20},
        "Processing": {"base": 120, "std": 25},
        "Packaging": {"base": 500, "std": 50},
        "Filling": {"base": 600, "std": 60},
        "Assembly": {"base": 80, "std": 15},
        "Machining": {"base": 40, "std": 10},
        "Testing": {"base": 200, "std": 30},
        "Utility": {"base": 50, "std": 10}
    }
    
    # Adjust production rate for different time periods
    if time_period == "Shift":
        period_factor = 8  # 8 hours per shift
    elif time_period == "Day":
        period_factor = 24  # 24 hours per day
    elif time_period == "Week":
        period_factor = 168  # 168 hours per week
    else:
        period_factor = 1  # default for Hour
    
    # Generate performance data for each period
    for i in range(num_periods):
        # Select equipment (with replacement)
        equipment = equipment_df.sample(1).iloc[0]
        equipment_id = equipment['equipment_id']
        
        # Get equipment category
        if 'equipment_type' in equipment.index:
            category = equipment['equipment_type']
        else:
            category = random.choice(equipment_categories)
        
        data["equipment_id"].append(equipment_id)
        
        # Assign work order (80% of records have work orders)
        if random.random() < 0.8:
            data["work_order_id"].append(random.choice(work_order_ids))
        else:
            data["work_order_id"].append("")
        
        # Assign shift ID based on time of day
        timestamp = period_starts[i]
        hour_of_day = timestamp.hour
        
        if time_period == "Shift":
            # Morning shift (6am-2pm), Afternoon shift (2pm-10pm), Night shift (10pm-6am)
            if 6 <= hour_of_day < 14:
                shift_id = shift_ids[0]  # Morning shift
            elif 14 <= hour_of_day < 22:
                shift_id = shift_ids[1]  # Afternoon shift
            else:
                shift_id = shift_ids[2]  # Night shift
        else:
            # For non-shift periods, use a simple rotation
            shift_idx = i % len(shift_ids)
            shift_id = shift_ids[shift_idx]
        
        data["shift_id"].append(shift_id)
        
        # Set timestamp
        data["timestamp"].append(timestamp.strftime("%Y-%m-%d %H:%M:%S"))
        
        # Set time period
        data["time_period"].append(time_period)
        
        # Get performance profile for this equipment category
        if category in performance_profiles:
            profile = performance_profiles[category]
        else:
            # Use default profile if category not found
            profile = performance_profiles[equipment_categories[0]]
        
        # Generate performance metrics with some time correlation
        # This creates more realistic patterns where consecutive periods have similar performance
        
        # Determine if this is a "bad day" (10% chance)
        bad_day = random.random() < 0.1
        
        # Generate availability percentage
        if bad_day:
            # Lower availability on "bad days"
            availability = max(0, min(100, 100 * random.normalvariate(
                profile["availability"]["base"] * 0.7,  # 30% reduction on bad days
                profile["availability"]["std"]
            )))
        else:
            availability = max(0, min(100, 100 * random.normalvariate(
                profile["availability"]["base"],
                profile["availability"]["std"]
            )))
        
        # Generate performance percentage
        if bad_day:
            # Lower performance on "bad days"
            performance = max(0, min(100, 100 * random.normalvariate(
                profile["performance"]["base"] * 0.8,  # 20% reduction on bad days
                profile["performance"]["std"]
            )))
        else:
            performance = max(0, min(100, 100 * random.normalvariate(
                profile["performance"]["base"],
                profile["performance"]["std"]
            )))
        
        # Generate quality percentage
        if bad_day:
            # Lower quality on "bad days"
            quality = max(0, min(100, 100 * random.normalvariate(
                profile["quality"]["base"] * 0.9,  # 10% reduction on bad days
                profile["quality"]["std"] * 1.5  # More variability on bad days
            )))
        else:
            quality = max(0, min(100, 100 * random.normalvariate(
                profile["quality"]["base"],
                profile["quality"]["std"]
            )))
        
        # Calculate OEE (Overall Equipment Effectiveness)
        oee = (availability * performance * quality) / 10000  # Convert from percentage
        
        # Round metrics to 1 decimal place
        availability = round(availability, 1)
        performance = round(performance, 1)
        quality = round(quality, 1)
        oee = round(oee, 1)
        
        data["availability_percent"].append(availability)
        data["performance_percent"].append(performance)
        data["quality_percent"].append(quality)
        data["oee_percent"].append(oee)
        
        # Calculate production counts based on availability, performance, and time period
        if category in production_rates:
            base_rate = production_rates[category]["base"]
            rate_std = production_rates[category]["std"]
        else:
            base_rate = 100
            rate_std = 20
        
        # Adjust production rate based on performance
        rate_factor = performance / 100
        
        # Add some random variation
        production_rate = random.normalvariate(base_rate * rate_factor, rate_std * rate_factor)
        
        # Scale by time period
        production_count = int(production_rate * period_factor * (availability / 100))
        
        # Calculate rejects based on quality percentage
        reject_rate = 1 - (quality / 100)
        reject_count = int(production_count * reject_rate)
        
        # Adjust production count to be gross production (including rejects)
        production_count += reject_count
        
        data["production_count"].append(production_count)
        data["reject_count"].append(reject_count)
        
        # Calculate downtime based on availability
        downtime_minutes = period_minutes * (1 - availability / 100)
        data["downtime_minutes"].append(round(downtime_minutes))
        
        # Calculate cycle time
        base_cycle_time = profile["cycle_time"]["base"]
        cycle_std = profile["cycle_time"]["std"]
        
        # Adjust cycle time based on performance (lower performance = higher cycle time)
        cycle_factor = 100 / performance
        cycle_time = random.normalvariate(base_cycle_time * cycle_factor, cycle_std)
        
        # Ensure cycle time is positive
        cycle_time = max(1, round(cycle_time))
        
        data["cycle_time_seconds"].append(cycle_time)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    
    print(f"Successfully generated {len(df)} production performance records.")
    print(f"Data saved to {output_file}")
    
    return df

def display_statistics(production_performance_df):
    """
    Display basic statistics about the generated production performance data
    
    Parameters:
    - production_performance_df: DataFrame containing production performance data
    """
    if production_performance_df is None or len(production_performance_df) == 0:
        print("No production performance data to analyze.")
        return
    
    print("\nProduction Performance Statistics:")
    print(f"Total performance records: {len(production_performance_df)}")
    
    # Time period distribution
    print("\nTime Period Distribution:")
    period_counts = production_performance_df['time_period'].value_counts()
    for period, count in period_counts.items():
        print(f"  {period}: {count} ({count/len(production_performance_df)*100:.1f}%)")
    
    # OEE statistics
    print("\nOEE Statistics:")
    print(f"  Average OEE: {production_performance_df['oee_percent'].mean():.1f}%")
    print(f"  Minimum OEE: {production_performance_df['oee_percent'].min():.1f}%")
    print(f"  Maximum OEE: {production_performance_df['oee_percent'].max():.1f}%")
    
    # OEE component statistics
    print("\nOEE Component Statistics:")
    print(f"  Average Availability: {production_performance_df['availability_percent'].mean():.1f}%")
    print(f"  Average Performance: {production_performance_df['performance_percent'].mean():.1f}%")
    print(f"  Average Quality: {production_performance_df['quality_percent'].mean():.1f}%")
    
    # Production and reject statistics
    total_production = production_performance_df['production_count'].sum()
    total_rejects = production_performance_df['reject_count'].sum()
    reject_rate = (total_rejects / total_production) * 100 if total_production > 0 else 0
    
    print("\nProduction Statistics:")
    print(f"  Total production: {total_production} units")
    print(f"  Total rejects: {total_rejects} units")
    print(f"  Overall reject rate: {reject_rate:.2f}%")
    print(f"  Average production per period: {production_performance_df['production_count'].mean():.1f} units")
    print(f"  Average rejects per period: {production_performance_df['reject_count'].mean():.1f} units")
    
    # Downtime statistics
    total_downtime = production_performance_df['downtime_minutes'].sum()
    avg_downtime = production_performance_df['downtime_minutes'].mean()
    
    print("\nDowntime Statistics:")
    print(f"  Total downtime: {total_downtime} minutes ({total_downtime/60:.1f} hours)")
    print(f"  Average downtime per period: {avg_downtime:.1f} minutes")
    
    # Cycle time statistics
    print("\nCycle Time Statistics:")
    print(f"  Average cycle time: {production_performance_df['cycle_time_seconds'].mean():.1f} seconds")
    print(f"  Minimum cycle time: {production_performance_df['cycle_time_seconds'].min():.1f} seconds")
    print(f"  Maximum cycle time: {production_performance_df['cycle_time_seconds'].max():.1f} seconds")
    
    # Equipment statistics
    equipment_count = production_performance_df['equipment_id'].nunique()
    print(f"\nUnique equipment tracked: {equipment_count}")
    
    # Top 5 equipment by production volume
    equipment_production = production_performance_df.groupby('equipment_id')['production_count'].sum().sort_values(ascending=False)
    
    print("\nTop 5 Equipment by Production Volume:")
    for equipment_id, production in equipment_production.head(5).items():
        print(f"  {equipment_id}: {production} units")
    
    # Bottom 5 equipment by OEE
    equipment_oee = production_performance_df.groupby('equipment_id')['oee_percent'].mean().sort_values()
    
    print("\nBottom 5 Equipment by Average OEE:")
    for equipment_id, oee in equipment_oee.head(5).items():
        print(f"  {equipment_id}: {oee:.1f}%")
    
    # Work order statistics
    wo_count = production_performance_df['work_order_id'].apply(lambda x: x != "").sum()
    wo_percentage = (wo_count / len(production_performance_df)) * 100
    
    print(f"\nRecords with work order association: {wo_count} ({wo_percentage:.1f}%)")
    
    # Time-based analysis
    production_performance_df['timestamp'] = pd.to_datetime(production_performance_df['timestamp'])
    
    # Group by day
    production_performance_df['day'] = production_performance_df['timestamp'].dt.date
    daily_oee = production_performance_df.groupby('day')['oee_percent'].mean()
    
    print("\nDaily Average OEE (first 7 days):")
    for day, oee in daily_oee.head(7).items():
        print(f"  {day}: {oee:.1f}%")
    
    # Shift performance comparison
    shift_performance = production_performance_df.groupby('shift_id')[
        ['oee_percent', 'availability_percent', 'performance_percent', 'quality_percent']
    ].mean()
    
    print("\nPerformance by Shift:")
    for shift_id, metrics in shift_performance.iterrows():
        print(f"  {shift_id}:")
        print(f"    OEE: {metrics['oee_percent']:.1f}%")
        print(f"    Availability: {metrics['availability_percent']:.1f}%")
        print(f"    Performance: {metrics['performance_percent']:.1f}%")
        print(f"    Quality: {metrics['quality_percent']:.1f}%")

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("data", exist_ok=True)
    
    # Load required data
    equipment_df = load_equipment_data()
    work_orders_df = load_work_orders_data()
    maintenance_df = load_maintenance_activities()
    equipment_states_df = load_equipment_states()
    
    if equipment_df is not None:
        # Generate production performance data
        production_performance_df = generate_production_performance(
            equipment_df,
            work_orders_df,
            maintenance_df,
            equipment_states_df,
            num_periods=1000,  # Generate 1000 performance records
            time_period="Hour",  # Hourly metrics
            output_file="data/production_performance.csv"
        )
        
        # Display statistics
        if production_performance_df is not None:
            display_statistics(production_performance_df)
            
            # Display sample data
            print("\nSample production performance data (first 5 records):")
            print(production_performance_df.head(5))

Successfully generated 1000 production performance records.
Data saved to data/production_performance.csv

Production Performance Statistics:
Total performance records: 1000

Time Period Distribution:
  Hour: 1000 (100.0%)

OEE Statistics:
  Average OEE: 72.1%
  Minimum OEE: 25.8%
  Maximum OEE: 98.7%

OEE Component Statistics:
  Average Availability: 88.2%
  Average Performance: 83.6%
  Average Quality: 97.0%

Production Statistics:
  Total production: 75408 units
  Total rejects: 1549 units
  Overall reject rate: 2.05%
  Average production per period: 75.4 units
  Average rejects per period: 1.5 units

Downtime Statistics:
  Total downtime: 7089 minutes (118.2 hours)
  Average downtime per period: 7.1 minutes

Cycle Time Statistics:
  Average cycle time: 109.3 seconds
  Minimum cycle time: 53.0 seconds
  Maximum cycle time: 202.0 seconds

Unique equipment tracked: 150

Top 5 Equipment by Production Volume:
  EQ-6D152291: 1323 units
  EQ-39407330: 1083 units
  EQ-720A372C: 1026 units
